In [1]:
import talib
import pandas as pd
import numpy as np
import yfinance as yf
from talib import MA_Type
import requests

In [2]:
#define capital (starting balance)
STARTING_BALANCE = 100000

In [3]:
# import the data from file
data=pd.read_csv('/Users/krutarth/Downloads/SPY and Sectoral ETFS_Final (1).csv')
data=data.rename(columns={"Dates":'time'})

/var/folders/l9/2w5b5dv946qbxxf5_vhp14v40000gn/T/ipykernel_40973/726622537.py:2: DtypeWarning: Columns (8,16,24,32,40,48,56,64,72) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('/Users/krutarth/Downloads/SPY and Sectoral ETFS_Final (1).csv')


In [4]:
data.dropna()

,time,Open,Close,High,Low,Value,Volume,Number Ticks,Dates.1,Open.1,...,Volume.10,Number Ticks.10,Dates.11,Open.11,Close.11,High.11,Low.11,Value.11,Volume.11,Number Ticks.11
0,5/19/22 9:31,388.970,389.650,389.700,388.960,120439024.0,309401,1891,5/19/22 9:31,338.000,...,17443.0,140.0,5/19/22 9:31,110.940,111.430,111.430,110.940,587036.813,5285.0,22.0
1,5/19/22 9:32,389.670,389.682,390.090,389.450,148440384.0,380869,2171,5/19/22 9:32,338.500,...,9941.0,70.0,5/19/22 9:32,111.455,111.400,111.491,111.240,529955.188,4760.0,10.0
2,5/19/22 9:33,389.750,390.430,390.610,389.675,204915056.0,525234,3212,5/19/22 9:33,338.954,...,25587.0,157.0,5/19/22 9:33,111.342,111.670,111.670,111.342,378633.156,3394.0,7.0
3,5/19/22 9:34,390.450,390.440,390.580,390.020,112328024.0,287817,1984,5/19/22 9:34,339.510,...,15862.0,123.0,5/19/22 9:34,111.700,111.710,111.850,111.660,714916.000,6400.0,30.0
4,5/19/22 9:35,390.430,390.540,390.630,389.930,161929824.0,414854,2521,5/19/22 9:35,339.145,...,17252.0,122.0,5/19/22 9:35,111.761,111.750,111.880,111.710,301063.813,2693.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16038,7/19/22 15:16,391.700,391.928,391.940,391.675,46968420.0,119865,699,8/1/22 9:53,369.475,...,4294.0,39.0,7/20/22 15:12,102.230,102.270,102.270,102.230,101844.125,996.0,4.0
16039,7/19/22 15:17,391.920,391.950,392.010,391.880,35570772.0,90754,562,8/1/22 9:54,369.670,...,9220.0,62.0,7/20/22 15:13,102.195,102.210,102.210,102.180,55391.188,542.0,4.0
16040,7/19/22 15:18,391.940,392.060,392.090,391.940,29061458.0,74130,424,8/1/22 9:55,369.709,...,4932.0,38.0,7/20/22 15:14,102.250,102.220,102.250,102.220,73201.539,716.0,6.0
16041,7/19/22 15:19,392.056,392.020,392.056,391.920,22723960.0,57970,401,8/1/22 9:56,369.570,...,30151.0,252.0,7/20/22 15:15,102.210,102.271,102.290,102.210,330536.156,3232.0,4.0


In [5]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'BUY' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class BBStrategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
    def trading_allowed(self):
        for pos in self.positions:
            if pos.status == 'open':
                return False
        
        return True
        
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.BBsignal == 'BUY' and self.trading_allowed():
                sl = data.Close - 1 * data.sd
                tp = data.Close + 1.2 * data.sd
                self.add_position(Position(data.time, data.Close, data.BBsignal, self.volume, sl, tp))
                
            elif data.BBsignal == 'SELL' and self.trading_allowed():
                sl = data.Close + 1 * data.sd
                tp = data.Close - 1.2 * data.sd
                self.add_position(Position(data.time, data.Close, data.BBsignal, self.volume, sl, tp))
                
            for pos in self.positions:
                if pos.status == 'open':
                    if (pos.sl >= data.Close and pos.order_type == 'BUY'):
                        pos.close_position(data.time, pos.sl)
                    elif (pos.sl <= data.Close and pos.order_type == 'SELL'):
                        pos.close_position(data.time, pos.sl)
                    elif (pos.tp <= data.Close and pos.order_type == 'BUY'):
                        pos.close_position(data.time, pos.tp)
                    elif (pos.tp >= data.Close and pos.order_type == 'SELL'):
                        pos.close_position(data.time, pos.tp)
                        
        return self.get_positions_df()

class macd_Strategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
    def trading_allowed(self):
        for pos in self.positions:
            if pos.status == 'open':
                return False
        
        return True
        
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.macd_trade_signal == 'BUY' and self.trading_allowed():
                sl = data.Close - 1 * data.sd
                tp = data.Close + 1.2 * data.sd
                self.add_position(Position(data.time, data.Close, data.macd_trade_signal, self.volume, sl, tp))
                
            elif data.macd_trade_signal == 'SELL' and self.trading_allowed():
                sl = data.Close + 1 * data.sd
                tp = data.Close - 1.2 * data.sd
                self.add_position(Position(data.time, data.Close, data.macd_trade_signal, self.volume, sl, tp))
                
            for pos in self.positions:
                if pos.status == 'open':
                    if (pos.sl >= data.Close and pos.order_type == 'BUY'):
                        pos.close_position(data.time, pos.sl)
                    elif (pos.sl <= data.Close and pos.order_type == 'SELL'):
                        pos.close_position(data.time, pos.sl)
                    elif (pos.tp <= data.Close and pos.order_type == 'BUY'):
                        pos.close_position(data.time, pos.tp)
                    elif (pos.tp >= data.Close and pos.order_type == 'SELL'):
                        pos.close_position(data.time, pos.tp)
                        
        return self.get_positions_df()

# class Strategy defines trading logic and evaluates the backtest based on opened/closed positions
class RSI_Strategy:
    def __init__(self, df, starting_balance):
        self.starting_balance = starting_balance
        self.positions = []
        self.data = df
    
    # return backtest result
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
    
    # add Position class to list
    def add_position(self, position):
        self.positions.append(position)
        return True
    
    # close positions when stop loss or take profit is reached
    def close_tp_sl(self, data):
        for pos in self.positions:
                    if pos.status == 'open':
                        if (pos.sl >= data.Close and pos.order_type == 'BUY'):
                            pos.close_position(data.time, pos.sl)
                        elif (pos.sl <= data.Close and pos.order_type == 'SELL'):
                            pos.close_position(data.time, pos.sl)
                        elif (pos.tp <= data.Close and pos.order_type == 'BUY'):
                            pos.close_position(data.time, pos.tp)
                        elif (pos.tp >= data.Close and pos.order_type == 'SELL'):
                            pos.close_position(data.time, pos.tp)
                            
    # check for open positions
    def has_open_positions(self):
        for pos in self.positions:
            if pos.status == 'open':
                return True
        return False
    
    # strategy logic how positions should be opened/closed
    def logic(self, data):
        
        # if no position is open
        if not self.has_open_positions():
            
            # if RSI less then 30 -> BUY
            if data['rsi_14'] < 20:
                
                # Position variables
                open_datetime = data['time']
                open_price = data['Close']
                order_type = 'BUY'
                volume = 250 #volume fixed at 250
                sl = open_price - 2 * data['atr_14']
                tp = open_price + 2 * data['atr_14']
                
                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
            
            # if RSI greater than 70 -> SELL
            elif data['rsi_14'] > 90:
                
                # Position variables
                open_datetime = data['time']
                open_price = data['Close']
                order_type = 'SELL'
                volume = 250 #volume fixed at 250
                sl = open_price + 1 * data['atr_14']
                tp = open_price - 2 * data['atr_14']
                
                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
        
        
# logic
    def run(self):
        # data represents a moment in time while iterating through the backtest
        for i, data in self.data.iterrows():
            # close positions when stop loss or take profit is reached
            self.close_tp_sl(data)
            
            # strategy logic
            self.logic(data)
        
        return self.get_positions_df()
    
# Stochastic Strategy    
class STOCH_Strategy:
    def __init__(self, df, starting_balance):
        self.starting_balance = starting_balance
        self.positions = []
        self.data = df
    
    # return backtest result
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
    
    # add Position class to list
    def add_position(self, position):
        self.positions.append(position)
        return True
    
    # close positions when stop loss or take profit is reached
    def close_tp_sl(self, data):
        for pos in self.positions:
                    if pos.status == 'open':
                        if (pos.sl >= data.Close and pos.order_type == 'BUY'):
                            pos.close_position(data.time, pos.sl)
                        elif (pos.sl <= data.Close and pos.order_type == 'SELL'):
                            pos.close_position(data.time, pos.sl)
                        elif (pos.tp <= data.Close and pos.order_type == 'BUY'):
                            pos.close_position(data.time, pos.tp)
                        elif (pos.tp >= data.Close and pos.order_type == 'SELL'):
                            pos.close_position(data.time, pos.tp)
                            
    # check for open positions
    def has_open_positions(self):
        for pos in self.positions:
            if pos.status == 'open':
                return True
        return False
    
    # strategy logic how positions should be opened/closed
    def logic(self, data):
        
        # if no position is open
        if not self.has_open_positions():
            
            # if indicators less then 25 -> BUY
            if data['slowk'] < 25 and data['slowd'] < 25:
                
                # Position variables
                open_datetime = data['time']
                open_price = data['Close']
                order_type = 'BUY'
                volume = 250 #volume fixed at 250
                sl = open_price - 2 * data['sd']
                tp = open_price + 2 * data['sd']
                
                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
            
            # if indicators greater than 80 -> SELL
            elif data['slowk'] > 80 and data['slowd'] > 80:
                
                # Position variables
                open_datetime = data['time']
                open_price = data['Close']
                order_type = 'SELL'
                volume = 250 #volume fixed at 250
                sl = open_price + 1 * data['sd']
                tp = open_price - 2 * data['sd']
                
                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
        
        
# logic
    def run(self):
        # data represents a moment in time while iterating through the backtest
        for i, data in self.data.iterrows():
            # close positions when stop loss or take profit is reached
            self.close_tp_sl(data)
            
            # strategy logic
            self.logic(data)
        
        return self.get_positions_df()

# Generate MFI Strategy
class MFI_Strategy:
    def __init__(self, df, starting_balance):
        self.starting_balance = starting_balance
        self.positions = []
        self.data = df
    
    # return backtest result
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
    
    # add Position class to list
    def add_position(self, position):
        self.positions.append(position)
        return True
    
    # close positions when stop loss or take profit is reached
    def close_tp_sl(self, data):
        for pos in self.positions:
                    if pos.status == 'open':
                        if (pos.sl >= data.Close and pos.order_type == 'BUY'):
                            pos.close_position(data.time, pos.sl)
                        elif (pos.sl <= data.Close and pos.order_type == 'SELL'):
                            pos.close_position(data.time, pos.sl)
                        elif (pos.tp <= data.Close and pos.order_type == 'BUY'):
                            pos.close_position(data.time, pos.tp)
                        elif (pos.tp >= data.Close and pos.order_type == 'SELL'):
                            pos.close_position(data.time, pos.tp)
                            
    # check for open positions
    def has_open_positions(self):
        for pos in self.positions:
            if pos.status == 'open':
                return True
        return False
    
    # strategy logic how positions should be opened/closed
    def logic(self, data):
        
        # if no position is open
        if not self.has_open_positions():
            
            # if indicators less then 25 -> BUY
            if data['MFI'] < 25:
                
                # Position variables
                open_datetime = data['time']
                open_price = data['Close']
                order_type = 'BUY'
                volume = 250 #volume fixed at 250
                sl = open_price - 2 * data['sd']
                tp = open_price + 3 * data['sd']
                
                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
            
            # if indicators greater than 70 -> SELL
            elif data['MFI'] > 70 and data['slowd'] > 80:
                
                # Position variables
                open_datetime = data['time']
                open_price = data['Close']
                order_type = 'SELL'
                volume = 250 #volume fixed at 250
                sl = open_price + 1 * data['sd']
                tp = open_price - 2 * data['sd']
                
                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
        
        
# logic
    def run(self):
        # data represents a moment in time while iterating through the backtest
        for i, data in self.data.iterrows():
            # close positions when stop loss or take profit is reached
            self.close_tp_sl(data)
            
            # strategy logic
            self.logic(data)
        
        return self.get_positions_df()

In [6]:
etf1 = data.iloc[:,:8]

In [7]:
etf1

,time,Open,Close,High,Low,Value,Volume,Number Ticks
0,5/19/22 9:31,388.970,389.650,389.70,388.960,1.204390e+08,309401,1891
1,5/19/22 9:32,389.670,389.682,390.09,389.450,1.484404e+08,380869,2171
2,5/19/22 9:33,389.750,390.430,390.61,389.675,2.049151e+08,525234,3212
3,5/19/22 9:34,390.450,390.440,390.58,390.020,1.123280e+08,287817,1984
4,5/19/22 9:35,390.430,390.540,390.63,389.930,1.619298e+08,414854,2521
...,...,...,...,...,...,...,...,...
52772,11/30/22 15:58,406.720,406.710,406.95,406.680,1.087078e+09,2672334,10891
52773,11/30/22 15:59,406.695,407.330,407.68,406.695,3.899521e+09,9573672,29260
52774,11/30/22 16:00,407.370,407.420,407.42,407.360,6.110610e+05,1500,13
52775,11/30/22 16:09,407.680,407.680,407.68,407.680,0.000000e+00,0,1


In [8]:
# Generate Stichastic Oscillator 
etf1['slowk'], etf1['slowd'] = talib.STOCH(etf1['High'], etf1['Low'], etf1['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf1['MFI'] = talib.MFI(etf1['High'], etf1['Low'], etf1['Close'], etf1['Volume'], timeperiod=14)

# generate bollinger bands
etf1['BBupper'], etf1['BBmiddle'], etf1['BBlower'] = talib.BBANDS(etf1['Close'], matype=MA_Type.T3)
etf1['sd']=etf1['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf1['BBsignal']=np.vectorize(trade_signal)(etf1['Close'], etf1['BBlower'],etf1['BBupper'])

# Generate MACD trading signals
etf1['macd'], etf1['macdsignal'], etf1['macdhist'] = talib.MACD(etf1['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf1['prev_macd']=etf1['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf1['macd_trade_signal']=np.vectorize(trade_signal)(etf1['prev_macd'], etf1['macd'],etf1['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf1['gain'] = (etf1['Close']- etf1['Open']).apply(lambda x: x if x > 0 else 0)
etf1['loss'] = (etf1['Close']- etf1['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf1['ema_gain'] = etf1['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf1['ema_loss'] = etf1['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf1['rs'] = etf1['ema_gain'] / etf1['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf1['rsi_14'] = 100 - (100 / (etf1['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf1['range'] = etf1['High'] - etf1['Low']

# calculating the average value of ranges
etf1['atr_14'] = etf1['range'].rolling(atr_period).mean()

etf1=etf1.dropna()

In [9]:
# run the backtest
bollinger_strategy_etf1 = BBStrategy(etf1, STARTING_BALANCE, 250) 
BBresult_etf1 = bollinger_strategy_etf1.run().dropna()
BBresult_etf1['Strategy'] = 'Bollinger Bands'
BBresult_etf1

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:06,389.950,BUY,250,372.422493,410.983008,5/27/22 9:54,410.983008,5258.252116,closed,105258.252116,Bollinger Bands
1,5/27/22 10:00,410.875,BUY,250,393.347493,431.908008,6/10/22 9:55,393.347493,-4381.876763,closed,100876.375353,Bollinger Bands
2,6/10/22 10:06,391.810,SELL,250,409.337507,370.776992,6/14/22 15:18,370.776992,5258.252116,closed,106134.627468,Bollinger Bands
3,6/14/22 15:20,370.900,SELL,250,388.427507,349.866992,6/24/22 15:43,388.427507,-4381.876763,closed,101752.750705,Bollinger Bands
4,6/24/22 15:48,388.330,BUY,250,370.802493,409.363008,7/29/22 10:00,409.363008,5258.252116,closed,107011.002821,Bollinger Bands
5,7/29/22 10:18,409.629,BUY,250,392.101493,430.662008,8/16/22 13:36,430.662008,5258.252116,closed,112269.254936,Bollinger Bands
6,8/16/22 13:43,430.700,BUY,250,413.172493,451.733008,8/22/22 14:10,413.172493,-4381.876763,closed,107887.378173,Bollinger Bands
7,8/22/22 14:50,412.855,SELL,250,430.382507,391.821992,9/1/22 10:02,391.821992,5258.252116,closed,113145.630289,Bollinger Bands
8,9/1/22 10:17,390.625,BUY,250,373.097493,411.658008,9/12/22 10:25,411.658008,5258.252116,closed,118403.882404,Bollinger Bands
9,9/12/22 10:47,411.110,SELL,250,428.637507,390.076992,9/15/22 15:11,390.076992,5258.252116,closed,123662.134520,Bollinger Bands


In [10]:
macd_trd_strategy_etf1=macd_Strategy(etf1, STARTING_BALANCE, 250)
macd_trade_etf1 = macd_trd_strategy_etf1.run().dropna()
macd_trade_etf1['Strategy'] = 'MACD'
macd_trade_etf1

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:05,390.500,BUY,250,372.972493,411.533008,5/27/22 10:03,411.533008,5258.252116,closed,105258.252116,MACD
1,5/27/22 10:21,411.950,BUY,250,394.422493,432.983008,6/10/22 9:33,394.422493,-4381.876763,closed,100876.375353,MACD
2,6/10/22 9:37,394.158,BUY,250,376.630493,415.191008,6/13/22 10:42,376.630493,-4381.876763,closed,96494.498590,MACD
3,6/13/22 10:43,376.100,SELL,250,393.627507,355.066992,7/20/22 10:31,393.627507,-4381.876763,closed,92112.621827,MACD
4,7/20/22 10:32,393.990,BUY,250,376.462493,415.023008,8/3/22 14:19,415.023008,5258.252116,closed,97370.873942,MACD
5,8/3/22 14:20,415.160,BUY,250,397.632493,436.193008,8/30/22 10:45,397.632493,-4381.876763,closed,92988.997179,MACD
6,8/30/22 10:50,396.720,BUY,250,379.192493,417.753008,9/21/22 15:44,379.192493,-4381.876763,closed,88607.120416,MACD
7,9/21/22 15:46,378.730,BUY,250,361.202493,399.763008,9/27/22 13:40,361.202493,-4381.876763,closed,84225.243653,MACD
8,9/27/22 13:44,361.425,BUY,250,343.897493,382.458008,10/25/22 9:59,382.458008,5258.252116,closed,89483.495769,MACD
9,10/25/22 10:00,382.130,SELL,250,399.657507,361.096992,11/14/22 13:25,399.657507,-4381.876763,closed,85101.619006,MACD


In [11]:
# creating an instance of Strategy class
rsi_strategy_etf1 = RSI_Strategy(etf1, STARTING_BALANCE)

# running the backtest
RSI_run_etf1 = rsi_strategy_etf1.run()
RSI_run_etf1['Strategy'] = 'RSI'
RSI_run_etf1

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:40,392.760,SELL,250,393.303000,391.674000,5/19/22 11:56,391.674000,271.500000,closed,100271.500000,RSI
1,5/19/22 12:20,390.890,BUY,250,390.021000,391.759000,5/19/22 12:40,390.021000,-217.250000,closed,100054.250000,RSI
2,5/19/22 12:40,389.890,BUY,250,389.063000,390.717000,5/19/22 12:45,389.063000,-206.750000,closed,99847.500000,RSI
3,5/19/22 12:45,389.030,BUY,250,388.198714,389.861286,5/19/22 12:57,389.861286,207.821429,closed,100055.321429,RSI
4,5/19/22 15:20,391.032,BUY,250,389.996286,392.067714,5/19/22 15:28,389.996286,-258.928571,closed,99796.392857,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
631,11/30/22 12:20,393.800,BUY,250,393.400143,394.199857,11/30/22 12:52,394.199857,99.964286,closed,91019.625000,RSI
632,11/30/22 13:30,396.550,SELL,250,396.901000,395.848000,11/30/22 13:31,396.901000,-87.750000,closed,90931.875000,RSI
633,11/30/22 13:31,397.110,SELL,250,397.543857,396.242286,11/30/22 13:32,396.242286,216.928571,closed,91148.803571,RSI
634,11/30/22 15:34,406.145,SELL,250,406.366071,405.702857,11/30/22 15:35,406.366071,-55.267857,closed,91093.535714,RSI


In [12]:
# STOCH Strategy Run
STOCH_Strategy_etf1 = STOCH_Strategy(etf1, STARTING_BALANCE)
STOCH_run_etf1 = STOCH_Strategy_etf1.run()

In [13]:
STOCH_run_etf1['Strategy'] = 'Stochastic'
STOCH_run_etf1

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:05,390.500,SELL,250,408.027507,355.444986,5/27/22 9:30,408.027507,-4381.876763,closed,95618.123237,Stochastic
1,5/27/22 9:32,408.950,SELL,250,426.477507,373.894986,6/13/22 15:50,373.894986,8763.753526,closed,104381.876763,Stochastic
2,6/13/22 15:52,373.885,BUY,250,338.829986,408.940014,7/29/22 9:48,408.940014,8763.753526,closed,113145.630289,Stochastic
3,7/29/22 9:58,409.110,BUY,250,374.054986,444.165014,9/22/22 10:59,374.054986,-8763.753526,closed,104381.876763,Stochastic
4,9/22/22 11:00,374.010,BUY,250,338.954986,409.065014,None,NaN,NaN,open,NaN,Stochastic


In [14]:
# MFI Strategy Run

mfi_strategy_etf1 = MFI_Strategy(etf1, STARTING_BALANCE)
MFI_run_etf1 = mfi_strategy_etf1.run()
MFI_run_etf1['Strategy'] = 'MFI'
MFI_run_etf1

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:34,390.991,SELL,250,408.518507,355.935986,5/27/22 9:31,408.518507,-4381.876763,closed,95618.123237,MFI
1,5/27/22 9:35,408.905,SELL,250,426.432507,373.849986,6/13/22 15:50,373.849986,8763.753526,closed,104381.876763,MFI
2,6/13/22 15:51,373.420,BUY,250,338.364986,426.002521,8/12/22 15:06,426.002521,13145.630289,closed,117527.507052,MFI
3,8/12/22 15:07,426.090,SELL,250,443.617507,391.034986,9/1/22 10:13,391.034986,8763.753526,closed,126291.260578,MFI
4,9/1/22 10:14,390.950,BUY,250,355.894986,443.532521,10/11/22 10:03,355.894986,-8763.753526,closed,117527.507052,MFI
5,10/11/22 10:03,355.820,BUY,250,320.764986,408.402521,None,NaN,NaN,open,NaN,MFI


In [15]:
etf1_combined = pd.concat([MFI_run_etf1, RSI_run_etf1, macd_trade_etf1, BBresult_etf1], ignore_index=True, sort=True)
etf1_combined = etf1_combined.sort_values(by='open_datetime')

In [16]:
etf1_combined = etf1_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [17]:
etf1_combined['Ticker'] = 'SPY'

In [18]:
etf1_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
454,RSI,10/10/22 11:01,10/10/22 10:37,BUY,-264.392857,closed,SPY
455,RSI,10/10/22 12:50,10/10/22 12:21,BUY,-116.678571,closed,SPY
456,RSI,10/10/22 13:02,10/10/22 12:53,BUY,-105.107143,closed,SPY
457,RSI,10/10/22 13:12,10/10/22 13:02,BUY,100.428571,closed,SPY
453,RSI,10/10/22 9:52,10/10/22 9:33,BUY,-176.607143,closed,SPY
...,...,...,...,...,...,...,...
349,RSI,9/8/22 12:10,9/8/22 12:05,BUY,-180.571429,closed,SPY
350,RSI,9/8/22 12:32,9/8/22 12:10,BUY,258.214286,closed,SPY
351,RSI,9/8/22 13:58,9/8/22 13:56,BUY,143.964286,closed,SPY
352,RSI,9/9/22 12:12,9/9/22 12:05,BUY,141.607143,closed,SPY


In [165]:
etf1_combined.to_csv(r'/Users/krutarth/Downloads/spy_analysis.csv')

In [19]:
etf2 = data.iloc[:,8:16]

In [20]:
etf2 = etf2.dropna()

In [21]:
etf2 = etf2.rename(columns={'Dates.1': 'time', 'Open.1': 'Open', 'Close.1' : 'Close', 'Low.1' : 'Low', 
                            'Value.1' : 'Value', 'Volume.1' : 'Volume', 'Number Ticks.1' : 'Number Ticks', 'High.1':'High'})

In [22]:
etf2

,time,Open,Close,High,Low,Value,Volume,Number Ticks
0,5/19/22 9:31,338.000,337.630,338.000,337.630,84444.500,250.0,2.0
1,5/19/22 9:32,338.500,338.975,339.000,338.500,457253.750,1349.0,6.0
2,5/19/22 9:33,338.954,339.570,339.650,338.954,283406.156,835.0,7.0
3,5/19/22 9:34,339.510,339.418,339.520,339.418,157178.219,463.0,4.0
4,5/19/22 9:35,339.145,339.245,339.245,338.980,160065.438,472.0,4.0
...,...,...,...,...,...,...,...,...
43855,11/30/22 15:57,347.080,347.210,347.270,347.035,694056.000,1999.0,15.0
43856,11/30/22 15:58,347.160,347.205,347.330,347.160,709768.750,2044.0,17.0
43857,11/30/22 15:59,347.270,347.940,348.000,347.240,5732544.500,16481.0,41.0
43858,11/30/22 16:09,347.940,347.940,347.940,347.940,0.000,0.0,1.0


In [23]:
# Generate Stichastic Oscillator 
etf2['slowk'], etf2['slowd'] = talib.STOCH(etf2['High'], etf2['Low'], etf2['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf2['MFI'] = talib.MFI(etf2['High'], etf2['Low'], etf2['Close'], etf2['Volume'], timeperiod=14)

# generate bollinger bands
etf2['BBupper'], etf2['BBmiddle'], etf2['BBlower'] = talib.BBANDS(etf2['Close'], matype=MA_Type.T3)
etf2['sd']=etf2['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf2['BBsignal']=np.vectorize(trade_signal)(etf2['Close'], etf2['BBlower'],etf2['BBupper'])

# Generate MACD trading signals
etf2['macd'], etf2['macdsignal'], etf2['macdhist'] = talib.MACD(etf2['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf2['prev_macd']=etf2['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf2['macd_trade_signal']=np.vectorize(trade_signal)(etf2['prev_macd'], etf2['macd'],etf2['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf2['gain'] = (etf2['Close']- etf2['Open']).apply(lambda x: x if x > 0 else 0)
etf2['loss'] = (etf2['Close']- etf2['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf2['ema_gain'] = etf2['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf2['ema_loss'] = etf2['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf2['rs'] = etf2['ema_gain'] / etf2['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf2['rsi_14'] = 100 - (100 / (etf2['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf2['range'] = etf2['High'] - etf2['Low']

# calculating the average value of ranges
etf2['atr_14'] = etf2['range'].rolling(atr_period).mean()

etf2=etf2.dropna()

In [24]:
# run the backtest
bollinger_strategy_etf2 = BBStrategy(etf2, STARTING_BALANCE, 250) 
BBresult_etf2 = bollinger_strategy_etf2.run().dropna()
BBresult_etf2['Strategy'] = 'Bollinger Bands'
BBresult_etf2

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:09,338.610,BUY,250,317.45312,363.998256,5/27/22 15:59,363.998256,6347.063934,closed,106347.063934,Bollinger Bands
1,5/31/22 9:30,362.771,BUY,250,341.61412,388.159256,6/10/22 9:56,341.614120,-5289.219945,closed,101057.843989,Bollinger Bands
2,6/10/22 10:06,340.240,SELL,250,361.39688,314.851744,6/16/22 14:07,314.851744,6347.063934,closed,107404.907923,Bollinger Bands
3,6/16/22 14:14,314.958,SELL,250,336.11488,289.569744,6/24/22 9:30,336.114880,-5289.219945,closed,102115.687978,Bollinger Bands
4,6/24/22 9:34,337.639,SELL,250,358.79588,312.250744,7/22/22 9:49,358.795880,-5289.219945,closed,96826.468033,Bollinger Bands
5,7/22/22 9:56,357.250,BUY,250,336.09312,382.638256,8/10/22 15:37,382.638256,6347.063934,closed,103173.531967,Bollinger Bands
6,8/10/22 15:48,382.800,BUY,250,361.64312,408.188256,8/26/22 15:44,361.643120,-5289.219945,closed,97884.312022,Bollinger Bands
7,8/26/22 15:51,361.795,SELL,250,382.95188,336.406744,9/15/22 10:52,336.406744,6347.063934,closed,104231.375956,Bollinger Bands
8,9/15/22 11:08,335.685,SELL,250,356.84188,310.296744,9/29/22 13:59,310.296744,6347.063934,closed,110578.439890,Bollinger Bands
9,9/29/22 14:06,310.380,SELL,250,331.53688,284.991744,10/6/22 9:43,331.536880,-5289.219945,closed,105289.219945,Bollinger Bands


In [25]:
macd_trd_strategy_etf2=macd_Strategy(etf2, STARTING_BALANCE, 250)
macd_trade_etf2 = macd_trd_strategy_etf2.run().dropna()
macd_trade_etf2['Strategy'] = 'MACD'
macd_trade_etf2

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:06,339.680,BUY,250,318.52312,365.068256,6/1/22 9:30,365.068256,6347.063934,closed,106347.063934,MACD
1,6/1/22 9:31,366.370,BUY,250,345.21312,391.758256,6/10/22 9:30,345.213120,-5289.219945,closed,101057.843989,MACD
2,6/10/22 9:38,343.638,BUY,250,322.48112,369.026256,6/13/22 15:13,322.481120,-5289.219945,closed,95768.624044,MACD
3,6/13/22 15:16,323.420,BUY,250,302.26312,348.808256,7/20/22 9:41,348.808256,6347.063934,closed,102115.687978,MACD
4,7/20/22 9:42,348.641,BUY,250,327.48412,374.029256,8/3/22 10:05,374.029256,6347.063934,closed,108462.751912,MACD
5,8/3/22 10:06,374.540,BUY,250,353.38312,399.928256,8/30/22 10:31,353.383120,-5289.219945,closed,103173.531967,MACD
6,8/30/22 10:32,352.390,SELL,250,373.54688,327.001744,9/22/22 9:37,327.001744,6347.063934,closed,109520.595901,MACD
7,9/22/22 9:38,326.310,SELL,250,347.46688,300.921744,10/11/22 10:03,300.921744,6347.063934,closed,115867.659836,MACD
8,10/11/22 10:05,300.680,BUY,250,279.52312,326.068256,10/25/22 9:30,326.068256,6347.063934,closed,122214.723770,MACD
9,10/25/22 9:31,326.508,BUY,250,305.35112,351.896256,11/4/22 11:51,305.351120,-5289.219945,closed,116925.503825,MACD


In [26]:
# creating an instance of Strategy class
rsi_strategy_etf2 = RSI_Strategy(etf2, STARTING_BALANCE)

# running the backtest
RSI_run_etf2 = rsi_strategy_etf2.run()
RSI_run_etf2['Strategy'] = 'RSI'
RSI_run_etf2

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:27,337.730,BUY,250,337.130571,338.329429,5/19/22 10:32,338.329429,149.857143,closed,100149.857143,RSI
1,5/19/22 11:43,342.665,SELL,250,342.928714,342.137571,5/19/22 11:49,342.928714,-65.928571,closed,100083.928571,RSI
2,5/19/22 12:20,340.620,BUY,250,340.310429,340.929571,5/19/22 12:22,340.929571,77.392857,closed,100161.321429,RSI
3,5/19/22 12:48,338.400,BUY,250,337.999429,338.800571,5/19/22 12:57,338.800571,100.142857,closed,100261.464286,RSI
4,5/19/22 13:10,337.240,BUY,250,336.858286,337.621714,5/19/22 13:18,337.621714,95.428571,closed,100356.892857,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
2170,11/30/22 12:57,332.190,BUY,250,332.120000,332.260000,11/30/22 12:58,332.120000,-17.500000,closed,101870.785714,RSI
2171,11/30/22 12:58,332.045,BUY,250,331.975000,332.115000,11/30/22 12:59,332.115000,17.500000,closed,101888.285714,RSI
2172,11/30/22 13:30,334.170,SELL,250,334.437643,333.634714,11/30/22 13:31,334.437643,-66.910714,closed,101821.375000,RSI
2173,11/30/22 13:31,335.460,SELL,250,335.773357,334.833286,11/30/22 13:32,334.833286,156.678571,closed,101978.053571,RSI


In [27]:
# STOCH Strategy Run
STOCH_Strategy_etf2 = STOCH_Strategy(etf2, STARTING_BALANCE)
STOCH_run_etf2 = STOCH_Strategy_etf2.run()
STOCH_run_etf2['Strategy'] = 'Stochastic'
STOCH_run_etf2

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:06,339.68,SELL,250,360.83688,297.36624,5/27/22 10:21,360.83688,-5289.219945,closed,94710.780055,Stochastic
1,5/27/22 10:23,361.08,SELL,250,382.23688,318.76624,6/16/22 10:04,318.76624,10578.439890,closed,105289.219945,Stochastic
2,6/16/22 10:04,318.65,BUY,250,276.33624,360.96376,7/28/22 9:31,360.96376,10578.439890,closed,115867.659836,Stochastic
3,7/28/22 9:41,359.64,BUY,250,317.32624,401.95376,9/23/22 12:28,317.32624,-10578.439890,closed,105289.219945,Stochastic
4,9/23/22 12:29,317.00,BUY,250,274.68624,359.31376,None,NaN,NaN,open,NaN,Stochastic


In [28]:
# MFI Strategy Run

mfi_strategy_etf2 = MFI_Strategy(etf2, STARTING_BALANCE)
MFI_run_etf2 = mfi_strategy_etf2.run()
MFI_run_etf2['Strategy'] = 'MFI'
MFI_run_etf2

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:06,339.680,SELL,250,360.83688,297.366240,5/27/22 10:21,360.836880,-5289.219945,closed,94710.780055,MFI
1,5/27/22 10:23,361.080,SELL,250,382.23688,318.766240,6/16/22 10:04,318.766240,10578.439890,closed,105289.219945,MFI
2,6/16/22 10:04,318.650,BUY,250,276.33624,382.120639,8/8/22 10:02,382.120639,15867.659836,closed,121156.879781,MFI
3,8/8/22 10:14,380.975,BUY,250,338.66124,444.445639,9/6/22 10:02,338.661240,-10578.439890,closed,110578.439890,MFI
4,9/6/22 10:02,338.440,BUY,250,296.12624,401.910639,10/13/22 9:30,296.126240,-10578.439890,closed,100000.000000,MFI
5,10/13/22 9:30,293.060,BUY,250,250.74624,356.530639,None,NaN,NaN,open,NaN,MFI


In [29]:
etf2_combined = pd.concat([MFI_run_etf2, RSI_run_etf2, macd_trade_etf2, BBresult_etf2], ignore_index=True, sort=True)
etf2_combined = etf2_combined.sort_values(by='open_datetime')

In [30]:
etf2 = etf2.rename(columns={'Dates.1': 'time', 'Open.1': 'Open', 'Close.1' : 'Close', 'Low.1' : 'Low', 
                            'Value.1' : 'Value', 'Volume.1' : 'Volume', 'Number Ticks.1' : 'Number Ticks', 'High.1':'High'})

In [31]:
etf2_combined = etf2_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [32]:
etf2_combined['Ticker'] = 'VGT'

In [33]:
etf2_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
1721,RSI,10/10/22 10:39,10/10/22 10:37,BUY,-119.035714,closed,VGT
1722,RSI,10/10/22 10:54,10/10/22 10:39,BUY,-132.607143,closed,VGT
1723,RSI,10/10/22 12:23,10/10/22 12:21,BUY,-39.285714,closed,VGT
1724,RSI,10/10/22 12:28,10/10/22 12:23,BUY,-38.857143,closed,VGT
1725,RSI,10/10/22 12:30,10/10/22 12:28,BUY,41.714286,closed,VGT
...,...,...,...,...,...,...,...
1432,RSI,9/9/22 14:53,9/9/22 14:52,SELL,30.500000,closed,VGT
1433,RSI,9/9/22 15:58,9/9/22 15:54,BUY,-47.428571,closed,VGT
1425,RSI,9/9/22 9:46,9/9/22 9:45,BUY,74.964286,closed,VGT
1426,RSI,9/9/22 9:47,9/9/22 9:46,BUY,69.607143,closed,VGT


In [180]:
etf2_combined.iloc[15:25]

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
2203,Bollinger Bands,10/25/22 15:26,10/10/22 9:43,SELL,-5289.219945,closed,VGT
1734,RSI,10/11/22 10:03,10/11/22 10:00,BUY,-112.571429,closed,VGT
2189,MACD,10/25/22 9:30,10/11/22 10:05,BUY,6347.063934,closed,VGT
1735,RSI,10/11/22 12:23,10/11/22 12:20,SELL,70.535714,closed,VGT
1736,RSI,10/11/22 13:16,10/11/22 13:15,BUY,-43.000000,closed,VGT
1737,RSI,10/11/22 13:17,10/11/22 13:16,BUY,47.642857,closed,VGT
1738,RSI,10/11/22 13:18,10/11/22 13:17,BUY,48.357143,closed,VGT
1739,RSI,10/11/22 14:45,10/11/22 14:44,BUY,-71.071429,closed,VGT
1740,RSI,10/11/22 14:46,10/11/22 14:45,BUY,-80.000000,closed,VGT
1741,RSI,10/11/22 14:48,10/11/22 14:46,BUY,-87.857143,closed,VGT


In [166]:
etf2_combined.to_csv(r'/Users/krutarth/Downloads/vgt_analysis.csv')

In [35]:
etf3 = data.iloc[:,16:24]

In [36]:
etf3 = etf3.rename(columns={'Dates.2': 'time', 'Open.2': 'Open', 'Close.2' : 'Close', 'Low.2' : 'Low', 
                            'Value.2' : 'Value', 'Volume.2' : 'Volume', 'Number Ticks.2' : 'Number Ticks', 'High.2':'High'})

In [37]:
etf3 = etf3.dropna()

In [38]:
# Generate Stichastic Oscillator 
etf3['slowk'], etf3['slowd'] = talib.STOCH(etf3['High'], etf3['Low'], etf3['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf3['MFI'] = talib.MFI(etf3['High'], etf3['Low'], etf3['Close'], etf3['Volume'], timeperiod=14)

# generate bollinger bands
etf3['BBupper'], etf3['BBmiddle'], etf3['BBlower'] = talib.BBANDS(etf3['Close'], matype=MA_Type.T3)
etf3['sd']=etf3['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf3['BBsignal']=np.vectorize(trade_signal)(etf3['Close'], etf3['BBlower'],etf3['BBupper'])

# Generate MACD trading signals
etf3['macd'], etf3['macdsignal'], etf3['macdhist'] = talib.MACD(etf3['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf3['prev_macd']=etf3['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf3['macd_trade_signal']=np.vectorize(trade_signal)(etf3['prev_macd'], etf3['macd'],etf3['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf3['gain'] = (etf3['Close']- etf3['Open']).apply(lambda x: x if x > 0 else 0)
etf3['loss'] = (etf3['Close']- etf3['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf3['ema_gain'] = etf3['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf3['ema_loss'] = etf3['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf3['rs'] = etf3['ema_gain'] / etf3['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf3['rsi_14'] = 100 - (100 / (etf3['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf3['range'] = etf3['High'] - etf3['Low']

# calculating the average value of ranges
etf3['atr_14'] = etf3['range'].rolling(atr_period).mean()

etf3=etf3.dropna()

In [39]:
# run the backtest
bollinger_strategy_etf3 = BBStrategy(etf3, STARTING_BALANCE, 250) 
BBresult_etf3 = bollinger_strategy_etf3.run().dropna()
BBresult_etf3['Strategy'] = 'Bollinger Bands'
BBresult_etf3

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:27,233.010,BUY,250,225.471767,242.055879,5/27/22 9:49,242.055879,2261.469756,closed,102261.469756,Bollinger Bands
1,5/27/22 10:05,242.941,BUY,250,235.402767,251.986879,6/2/22 9:52,235.402767,-1884.558130,closed,100376.911626,Bollinger Bands
2,6/2/22 10:00,235.110,SELL,250,242.648233,226.064121,6/13/22 9:30,226.064121,2261.469756,closed,102638.381382,Bollinger Bands
3,6/13/22 9:37,226.010,SELL,250,233.548233,216.964121,6/23/22 15:31,233.548233,-1884.558130,closed,100753.823252,Bollinger Bands
4,6/23/22 15:46,233.920,SELL,250,241.458233,224.874121,7/8/22 9:57,241.458233,-1884.558130,closed,98869.265122,Bollinger Bands
5,7/8/22 10:02,241.730,SELL,250,249.268233,232.684121,7/14/22 9:30,232.684121,2261.469756,closed,101130.734878,Bollinger Bands
6,7/14/22 9:36,232.290,BUY,250,224.751767,241.335879,7/21/22 15:56,241.335879,2261.469756,closed,103392.204634,Bollinger Bands
7,7/22/22 9:30,242.400,SELL,250,249.938233,233.354121,8/15/22 10:12,249.938233,-1884.558130,closed,101507.646504,Bollinger Bands
8,8/15/22 10:51,249.050,BUY,250,241.511767,258.095879,8/23/22 9:33,241.511767,-1884.558130,closed,99623.088374,Bollinger Bands
9,8/23/22 9:38,240.420,BUY,250,232.881767,249.465879,8/30/22 10:46,232.881767,-1884.558130,closed,97738.530244,Bollinger Bands


In [40]:
macd_trd_strategy_etf3=macd_Strategy(etf3, STARTING_BALANCE, 250)
macd_trade_etf3 = macd_trd_strategy_etf3.run().dropna()
macd_trade_etf3['Strategy'] = 'MACD'
macd_trade_etf3

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:12,232.470,BUY,250,224.931767,241.515879,5/26/22 15:49,241.515879,2261.469756,closed,102261.469756,MACD
1,5/26/22 15:50,241.640,BUY,250,234.101767,250.685879,6/9/22 15:57,234.101767,-1884.558130,closed,100376.911626,MACD
2,6/9/22 16:14,233.950,BUY,250,226.411767,242.995879,6/13/22 9:30,226.411767,-1884.558130,closed,98492.353496,MACD
3,6/13/22 9:37,226.010,BUY,250,218.471767,235.055879,6/16/22 9:38,218.471767,-1884.558130,closed,96607.795366,MACD
4,6/16/22 9:42,218.030,BUY,250,210.491767,227.075879,6/22/22 9:53,227.075879,2261.469756,closed,98869.265122,MACD
5,6/22/22 9:54,227.570,BUY,250,220.031767,236.615879,6/24/22 13:00,236.615879,2261.469756,closed,101130.734878,MACD
6,6/24/22 13:01,236.720,BUY,250,229.181767,245.765879,7/28/22 12:26,245.765879,2261.469756,closed,103392.204634,MACD
7,7/28/22 12:27,245.730,BUY,250,238.191767,254.775879,8/26/22 14:24,238.191767,-1884.558130,closed,101507.646504,MACD
8,8/26/22 14:27,238.130,BUY,250,230.591767,247.175879,9/1/22 9:34,230.591767,-1884.558130,closed,99623.088374,MACD
9,9/1/22 9:39,230.660,BUY,250,223.121767,239.705879,9/8/22 15:54,239.705879,2261.469756,closed,101884.558130,MACD


In [41]:
# creating an instance of Strategy class
rsi_strategy_etf3 = RSI_Strategy(etf3, STARTING_BALANCE)

# running the backtest
RSI_run_etf3 = rsi_strategy_etf3.run()
RSI_run_etf3['Strategy'] = 'RSI'
RSI_run_etf3

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 13:08,233.520,BUY,250,233.490714,233.549286,5/19/22 13:10,233.490714,-7.321429,closed,99992.678571,RSI
1,5/19/22 13:10,233.399,BUY,250,233.369714,233.428286,5/19/22 13:12,233.428286,7.321429,closed,100000.000000,RSI
2,5/19/22 13:12,233.579,BUY,250,233.549714,233.608286,5/19/22 13:13,233.549714,-7.321429,closed,99992.678571,RSI
3,5/19/22 13:13,233.390,BUY,250,233.360714,233.419286,5/19/22 13:16,233.360714,-7.321429,closed,99985.357143,RSI
4,5/19/22 13:16,233.290,BUY,250,233.260714,233.319286,5/19/22 13:18,233.319286,7.321429,closed,99992.678571,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
3741,11/30/22 15:16,252.600,SELL,250,252.644214,252.511571,11/30/22 15:17,252.644214,-11.053571,closed,102487.696429,RSI
3742,11/30/22 15:17,252.700,SELL,250,252.734571,252.630857,11/30/22 15:19,252.734571,-8.642857,closed,102479.053571,RSI
3743,11/30/22 15:19,252.750,SELL,250,252.790429,252.669143,11/30/22 15:21,252.790429,-10.107143,closed,102468.946429,RSI
3744,11/30/22 15:21,252.860,SELL,250,252.907929,252.764143,11/30/22 15:25,252.907929,-11.982143,closed,102456.964286,RSI


In [42]:
# STOCH Strategy Run
STOCH_Strategy_etf3 = STOCH_Strategy(etf3, STARTING_BALANCE)
STOCH_run_etf3 = STOCH_Strategy_etf3.run()
STOCH_run_etf3['Strategy'] = 'Stochastic'
STOCH_run_etf3

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:19,233.09,SELL,250,240.628233,218.013535,5/25/22 10:34,240.628233,-1884.558130,closed,98115.441870,Stochastic
1,5/25/22 10:34,240.64,SELL,250,248.178233,225.563535,6/13/22 9:30,225.563535,3769.116259,closed,101884.558130,Stochastic
2,6/13/22 9:30,225.12,BUY,250,210.043535,240.196465,6/28/22 9:35,240.196465,3769.116259,closed,105653.674389,Stochastic
3,6/28/22 9:35,240.22,SELL,250,247.758233,225.143535,8/10/22 15:45,247.758233,-1884.558130,closed,103769.116259,Stochastic
4,8/10/22 15:45,247.78,SELL,250,255.318233,232.703535,8/30/22 10:47,232.703535,3769.116259,closed,107538.232519,Stochastic
5,8/30/22 10:47,232.67,BUY,250,217.593535,247.746465,11/22/22 15:31,247.746465,3769.116259,closed,111307.348778,Stochastic
6,11/22/22 15:34,247.84,SELL,250,255.378233,232.763535,None,NaN,NaN,open,NaN,Stochastic


In [43]:
# MFI Strategy Run

mfi_strategy_etf3 = MFI_Strategy(etf3, STARTING_BALANCE)
MFI_run_etf3 = mfi_strategy_etf3.run()
MFI_run_etf3['Strategy'] = 'MFI'
MFI_run_etf3

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:25,232.965,SELL,250,240.503233,217.888535,5/25/22 10:33,240.503233,-1884.558130,closed,98115.441870,MFI
1,5/25/22 10:39,240.960,SELL,250,248.498233,225.883535,6/13/22 9:30,225.883535,3769.116259,closed,101884.558130,MFI
2,6/13/22 9:30,225.120,BUY,250,210.043535,247.734698,8/10/22 15:43,247.734698,5653.674389,closed,107538.232519,MFI
3,8/10/22 15:43,247.740,SELL,250,255.278233,232.663535,8/30/22 13:23,232.663535,3769.116259,closed,111307.348778,MFI
4,8/30/22 13:23,232.639,BUY,250,217.562535,255.253698,None,NaN,NaN,open,NaN,MFI


In [44]:
etf3_combined = pd.concat([MFI_run_etf3, RSI_run_etf3, macd_trade_etf3, BBresult_etf3], ignore_index=True, sort=True)
etf3_combined = etf3_combined.sort_values(by='open_datetime')

In [45]:
etf3_combined = etf3_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [46]:
etf3_combined['Ticker'] = 'VHT'

In [47]:
etf3_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
2816,RSI,10/10/22 10:51,10/10/22 10:49,BUY,14.107143,closed,VHT
2817,RSI,10/10/22 11:52,10/10/22 11:50,SELL,-5.696429,closed,VHT
2818,RSI,10/10/22 11:53,10/10/22 11:52,SELL,11.392857,closed,VHT
2819,RSI,10/10/22 12:00,10/10/22 11:53,SELL,11.392857,closed,VHT
2820,RSI,10/10/22 12:07,10/10/22 12:00,SELL,-5.696429,closed,VHT
...,...,...,...,...,...,...,...
2213,RSI,9/12/22 9:30,9/9/22 15:59,BUY,17.428571,closed,VHT
2176,RSI,9/9/22 9:33,9/9/22 9:30,BUY,22.464286,closed,VHT
2177,RSI,9/9/22 9:34,9/9/22 9:33,BUY,22.464286,closed,VHT
2178,RSI,9/9/22 9:48,9/9/22 9:34,BUY,22.928571,closed,VHT


In [167]:
etf3_combined.to_csv(r'/Users/krutarth/Downloads/vht_analysis.csv')

In [48]:
etf4 = data.iloc[:,24:32]

In [49]:
etf4 = etf4.rename(columns={'Dates.3': 'time', 'Open.3': 'Open', 'Close.3' : 'Close', 'Low.3' : 'Low', 
                            'Value.3' : 'Value', 'Volume.3' : 'Volume', 'Number Ticks.3' : 'Number Ticks', 'High.3':'High'})

In [50]:
etf4 = etf4.dropna()

In [51]:
# Generate Stichastic Oscillator 
etf4['slowk'], etf4['slowd'] = talib.STOCH(etf4['High'], etf4['Low'], etf4['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf4['MFI'] = talib.MFI(etf4['High'], etf4['Low'], etf4['Close'], etf4['Volume'], timeperiod=14)

# generate bollinger bands
etf4['BBupper'], etf4['BBmiddle'], etf4['BBlower'] = talib.BBANDS(etf4['Close'], matype=MA_Type.T3)
etf4['sd']=etf4['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf4['BBsignal']=np.vectorize(trade_signal)(etf4['Close'], etf4['BBlower'],etf4['BBupper'])

# Generate MACD trading signals
etf4['macd'], etf4['macdsignal'], etf4['macdhist'] = talib.MACD(etf4['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf4['prev_macd']=etf4['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf4['macd_trade_signal']=np.vectorize(trade_signal)(etf4['prev_macd'], etf4['macd'],etf4['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf4['gain'] = (etf4['Close']- etf4['Open']).apply(lambda x: x if x > 0 else 0)
etf4['loss'] = (etf4['Close']- etf4['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf4['ema_gain'] = etf4['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf4['ema_loss'] = etf4['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf4['rs'] = etf4['ema_gain'] / etf4['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf4['rsi_14'] = 100 - (100 / (etf4['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf4['range'] = etf4['High'] - etf4['Low']

# calculating the average value of ranges
etf4['atr_14'] = etf4['range'].rolling(atr_period).mean()

etf4=etf4.dropna()

In [52]:
# run the backtest
bollinger_strategy_etf4 = BBStrategy(etf4, STARTING_BALANCE, 250) 
BBresult_etf4 = bollinger_strategy_etf4.run().dropna()
BBresult_etf4['Strategy'] = 'Bollinger Bands'
BBresult_etf4

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:43,237.340,BUY,250,222.212107,255.493471,5/27/22 15:59,255.493471,4538.367818,closed,104538.367818,Bollinger Bands
1,5/31/22 9:30,256.380,SELL,250,271.507893,238.226529,6/13/22 9:30,238.226529,4538.367818,closed,109076.735635,Bollinger Bands
2,6/13/22 9:44,232.737,SELL,250,247.864893,214.583529,7/19/22 14:01,247.864893,-3781.973181,closed,105294.762454,Bollinger Bands
3,7/19/22 14:16,248.160,BUY,250,233.032107,266.313471,7/29/22 9:50,266.313471,4538.367818,closed,109833.130272,Bollinger Bands
4,7/29/22 9:58,266.420,BUY,250,251.292107,284.573471,8/15/22 15:23,284.573471,4538.367818,closed,114371.498089,Bollinger Bands
5,8/15/22 15:43,284.476,BUY,250,269.348107,302.629471,8/22/22 13:35,269.348107,-3781.973181,closed,110589.524908,Bollinger Bands
6,8/22/22 13:42,269.390,SELL,250,284.517893,251.236529,9/22/22 9:33,251.236529,4538.367818,closed,115127.892726,Bollinger Bands
7,9/22/22 9:47,249.865,SELL,250,264.992893,231.711529,10/10/22 12:39,231.711529,4538.367818,closed,119666.260544,Bollinger Bands
8,10/10/22 13:20,231.570,SELL,250,246.697893,213.416529,10/26/22 11:06,246.697893,-3781.973181,closed,115884.287362,Bollinger Bands
9,10/26/22 12:00,247.240,BUY,250,232.112107,265.393471,11/2/22 15:20,232.112107,-3781.973181,closed,112102.314181,Bollinger Bands


In [53]:
macd_trd_strategy_etf4=macd_Strategy(etf4, STARTING_BALANCE, 250)
macd_trade_etf4 = macd_trd_strategy_etf4.run().dropna()
macd_trade_etf4['Strategy'] = 'MACD'
macd_trade_etf4

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:35,239.190,BUY,250,224.062107,257.343471,5/31/22 12:40,257.343471,4538.367818,closed,104538.367818,MACD
1,5/31/22 12:45,258.000,BUY,250,242.872107,276.153471,6/10/22 10:00,242.872107,-3781.973181,closed,100756.394636,MACD
2,6/10/22 10:01,242.450,SELL,250,257.577893,224.296529,6/16/22 10:40,224.296529,4538.367818,closed,105294.762454,MACD
3,6/16/22 10:41,224.010,SELL,250,239.137893,205.856529,6/24/22 9:49,239.137893,-3781.973181,closed,101512.789273,MACD
4,6/24/22 9:50,239.271,SELL,250,254.398893,221.117529,7/21/22 10:01,254.398893,-3781.973181,closed,97730.816091,MACD
5,7/21/22 10:02,254.195,BUY,250,239.067107,272.348471,8/1/22 9:56,272.348471,4538.367818,closed,102269.183909,MACD
6,8/1/22 9:57,272.280,BUY,250,257.152107,290.433471,8/16/22 14:12,290.433471,4538.367818,closed,106807.551727,MACD
7,8/16/22 14:14,290.405,BUY,250,275.277107,308.558471,8/22/22 9:30,275.277107,-3781.973181,closed,103025.578545,MACD
8,8/22/22 9:57,271.299,BUY,250,256.171107,289.452471,9/1/22 9:30,256.171107,-3781.973181,closed,99243.605364,MACD
9,9/1/22 9:43,256.040,BUY,250,240.912107,274.193471,9/12/22 9:34,274.193471,4538.367818,closed,103781.973181,MACD


In [54]:
# creating an instance of Strategy class
rsi_strategy_etf4 = RSI_Strategy(etf4, STARTING_BALANCE)

# running the backtest
RSI_run_etf4 = rsi_strategy_etf4.run()
RSI_run_etf4['Strategy'] = 'RSI'
RSI_run_etf4

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:42,237.790,BUY,250,237.765714,237.814286,5/19/22 10:43,237.765714,-6.071429,closed,99993.928571,RSI
1,5/19/22 10:43,237.340,BUY,250,237.315714,237.364286,5/19/22 10:45,237.364286,6.071429,closed,100000.000000,RSI
2,5/19/22 10:45,237.754,BUY,250,237.729714,237.778286,5/19/22 10:47,237.778286,6.071429,closed,100006.071429,RSI
3,5/19/22 10:47,238.003,BUY,250,237.978714,238.027286,5/19/22 10:49,237.978714,-6.071429,closed,100000.000000,RSI
4,5/19/22 10:49,237.670,BUY,250,237.645714,237.694286,5/19/22 10:52,237.694286,6.071429,closed,100006.071429,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
4079,11/30/22 13:58,240.875,SELL,250,240.992500,240.640000,11/30/22 14:02,240.992500,-29.375000,closed,101999.214286,RSI
4080,11/30/22 14:02,242.090,SELL,250,242.207500,241.855000,11/30/22 14:05,242.207500,-29.375000,closed,101969.839286,RSI
4081,11/30/22 14:05,242.230,SELL,250,242.345571,241.998857,11/30/22 14:13,242.345571,-28.892857,closed,101940.946429,RSI
4082,11/30/22 15:25,244.290,BUY,250,244.234143,244.345857,11/30/22 15:26,244.345857,13.964286,closed,101954.910714,RSI


In [55]:
# STOCH Strategy Run
STOCH_Strategy_etf4 = STOCH_Strategy(etf4, STARTING_BALANCE)
STOCH_run_etf4 = STOCH_Strategy_etf4.run()
STOCH_run_etf4['Strategy'] = 'Stochastic'
STOCH_run_etf4

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:32,238.11,BUY,250,207.854215,268.365785,7/29/22 14:31,268.365785,7563.946363,closed,107563.946363,Stochastic
1,7/29/22 14:36,268.47,SELL,250,283.597893,238.214215,8/15/22 11:50,283.597893,-3781.973181,closed,103781.973181,Stochastic
2,8/15/22 11:50,283.61,SELL,250,298.737893,253.354215,9/1/22 10:59,253.354215,7563.946363,closed,111345.919544,Stochastic
3,9/1/22 11:00,252.43,BUY,250,222.174215,282.685785,11/9/22 15:48,222.174215,-7563.946363,closed,103781.973181,Stochastic
4,11/9/22 15:49,222.19,BUY,250,191.934215,252.445785,None,NaN,NaN,open,NaN,Stochastic


In [56]:
# MFI Strategy Run

mfi_strategy_etf4 = MFI_Strategy(etf4, STARTING_BALANCE)
MFI_run_etf4 = mfi_strategy_etf4.run()
MFI_run_etf4['Strategy'] = 'MFI'
MFI_run_etf4

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:55,238.830,SELL,250,253.957893,208.574215,5/27/22 14:58,253.957893,-3781.973181,closed,96218.026819,MFI
1,5/27/22 14:58,253.971,SELL,250,269.098893,223.715215,6/16/22 10:44,223.715215,7563.946363,closed,103781.973181,MFI
2,6/16/22 10:49,223.330,BUY,250,193.074215,268.713678,7/29/22 15:05,268.713678,11345.919544,closed,115127.892726,MFI
3,7/29/22 15:13,268.760,SELL,250,283.887893,238.504215,8/15/22 12:15,283.887893,-3781.973181,closed,111345.919544,MFI
4,8/15/22 12:59,284.207,SELL,250,299.334893,253.951215,9/1/22 10:03,253.951215,7563.946363,closed,118909.865907,MFI
5,9/1/22 10:23,254.332,BUY,250,224.076215,299.715678,10/13/22 9:32,224.076215,-7563.946363,closed,111345.919544,MFI
6,10/13/22 9:32,223.490,BUY,250,193.234215,268.873678,None,NaN,NaN,open,NaN,MFI


In [57]:
etf4_combined = pd.concat([MFI_run_etf4, RSI_run_etf4, macd_trade_etf4, BBresult_etf4], ignore_index=True, sort=True)
etf4_combined = etf4_combined.sort_values(by='open_datetime')

In [58]:
etf4_combined = etf4_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [59]:
etf4_combined['Ticker'] = 'VCR'

In [60]:
etf4_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
3023,RSI,10/10/22 10:38,10/10/22 10:37,BUY,-23.821429,closed,VCR
4115,Bollinger Bands,10/26/22 11:06,10/10/22 13:20,SELL,-3781.973181,closed,VCR
3024,RSI,10/10/22 14:22,10/10/22 14:14,BUY,-15.535714,closed,VCR
3025,RSI,10/10/22 14:26,10/10/22 14:22,BUY,15.892857,closed,VCR
3026,RSI,10/10/22 14:31,10/10/22 14:26,BUY,15.321429,closed,VCR
...,...,...,...,...,...,...,...
2362,RSI,9/9/22 9:37,9/9/22 9:32,SELL,-24.642857,closed,VCR
2363,RSI,9/9/22 9:45,9/9/22 9:37,SELL,46.785714,closed,VCR
2364,RSI,9/9/22 9:46,9/9/22 9:45,SELL,-26.660714,closed,VCR
2365,RSI,9/9/22 9:47,9/9/22 9:46,SELL,-26.660714,closed,VCR


In [168]:
etf4_combined.to_csv(r'/Users/krutarth/Downloads/vcr_analysis.csv')

In [61]:
etf5 = data.iloc[:,32:40]

In [62]:
etf5 = etf5.rename(columns={'Dates.4': 'time', 'Open.4': 'Open', 'Close.4' : 'Close', 'Low.4' : 'Low', 
                            'Value.4' : 'Value', 'Volume.4' : 'Volume', 'Number Ticks.4' : 'Number Ticks', 'High.4':'High'})

In [63]:
etf5 = etf5.dropna()

In [64]:
# Generate Stichastic Oscillator 
etf5['slowk'], etf5['slowd'] = talib.STOCH(etf5['High'], etf5['Low'], etf5['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf5['MFI'] = talib.MFI(etf5['High'], etf5['Low'], etf5['Close'], etf5['Volume'], timeperiod=14)

# generate bollinger bands
etf5['BBupper'], etf5['BBmiddle'], etf5['BBlower'] = talib.BBANDS(etf5['Close'], matype=MA_Type.T3)
etf5['sd']=etf5['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf5['BBsignal']=np.vectorize(trade_signal)(etf5['Close'], etf5['BBlower'],etf5['BBupper'])

# Generate MACD trading signals
etf5['macd'], etf5['macdsignal'], etf5['macdhist'] = talib.MACD(etf5['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf5['prev_macd']=etf5['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf5['macd_trade_signal']=np.vectorize(trade_signal)(etf5['prev_macd'], etf5['macd'],etf5['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf5['gain'] = (etf5['Close']- etf5['Open']).apply(lambda x: x if x > 0 else 0)
etf5['loss'] = (etf5['Close']- etf5['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf5['ema_gain'] = etf5['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf5['ema_loss'] = etf5['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf5['rs'] = etf5['ema_gain'] / etf5['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf5['rsi_14'] = 100 - (100 / (etf5['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf5['range'] = etf5['High'] - etf5['Low']

# calculating the average value of ranges
etf5['atr_14'] = etf5['range'].rolling(atr_period).mean()

etf5=etf5.dropna()

In [65]:
# run the backtest
bollinger_strategy_etf5 = BBStrategy(etf5, STARTING_BALANCE, 250) 
BBresult_etf5 = bollinger_strategy_etf5.run().dropna()
BBresult_etf5['Strategy'] = 'Bollinger Bands'
BBresult_etf5

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:20,100.7865,SELL,250,107.786579,92.386405,6/14/22 15:08,92.386405,2100.023818,closed,102100.023818,Bollinger Bands
1,6/14/22 15:12,92.2300,BUY,250,85.229921,100.630095,7/20/22 12:18,100.630095,2100.023818,closed,104200.047637,Bollinger Bands
2,7/20/22 12:42,100.2200,BUY,250,93.219921,108.620095,9/1/22 11:47,93.219921,-1750.019849,closed,102450.027788,Bollinger Bands
3,9/1/22 12:03,93.3200,SELL,250,100.320079,84.919905,9/23/22 10:01,84.919905,2100.023818,closed,104550.051607,Bollinger Bands


In [66]:
macd_trd_strategy_etf5=macd_Strategy(etf5, STARTING_BALANCE, 250)
macd_trade_etf5 = macd_trd_strategy_etf5.run().dropna()
macd_trade_etf5['Strategy'] = 'MACD'
macd_trade_etf5

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:17,100.4350,BUY,250,93.434921,108.835095,6/13/22 15:03,93.434921,-1750.019849,closed,98249.980151,MACD
1,6/13/22 15:17,93.4500,BUY,250,86.449921,101.850095,8/10/22 12:00,101.850095,2100.023818,closed,100350.003970,MACD
2,8/10/22 12:01,101.8349,BUY,250,94.834821,110.234995,8/30/22 10:00,94.834821,-1750.019849,closed,98599.984121,MACD
3,8/30/22 10:04,94.9400,BUY,250,87.939921,103.340095,9/21/22 14:14,87.939921,-1750.019849,closed,96849.964272,MACD
4,9/21/22 14:16,88.1300,BUY,250,81.129921,96.530095,10/13/22 9:30,81.129921,-1750.019849,closed,95099.944424,MACD
5,10/13/22 9:42,80.1100,BUY,250,73.109921,88.510095,10/25/22 11:51,88.510095,2100.023818,closed,97199.968242,MACD
6,10/25/22 11:57,88.5000,SELL,250,95.500079,80.099905,11/3/22 9:44,80.099905,2100.023818,closed,99299.992061,MACD
7,11/3/22 9:49,79.9800,BUY,250,72.979921,88.380095,11/15/22 11:33,88.380095,2100.023818,closed,101400.015879,MACD


In [67]:
# creating an instance of Strategy class
rsi_strategy_etf5 = RSI_Strategy(etf5, STARTING_BALANCE)

# running the backtest
RSI_run_etf5 = rsi_strategy_etf5.run()
RSI_run_etf5['Strategy'] = 'RSI'
RSI_run_etf5

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:55,100.510,SELL,250,100.523493,100.483014,5/19/22 10:56,100.483014,6.746429,closed,100006.746429,RSI
1,5/19/22 11:33,100.520,SELL,250,100.577300,100.405400,5/19/22 11:34,100.577300,-14.325000,closed,99992.421429,RSI
2,5/19/22 11:34,100.610,SELL,250,100.655871,100.518257,5/19/22 11:35,100.655871,-11.467857,closed,99980.953571,RSI
3,5/19/22 11:35,100.670,SELL,250,100.724400,100.561200,5/19/22 11:36,100.724400,-13.600000,closed,99967.353571,RSI
4,5/19/22 11:36,100.740,SELL,250,100.794400,100.631200,5/19/22 11:37,100.794400,-13.600000,closed,99953.753571,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
3971,11/30/22 14:26,88.105,SELL,250,88.158157,87.998686,11/30/22 14:32,87.998686,26.578571,closed,102352.735714,RSI
3972,11/30/22 14:35,88.000,SELL,250,88.039871,87.920257,11/30/22 14:36,87.920257,19.935714,closed,102372.671429,RSI
3973,11/30/22 14:36,87.920,SELL,250,87.950143,87.859714,11/30/22 14:38,87.859714,15.071429,closed,102387.742857,RSI
3974,11/30/22 14:38,87.800,SELL,250,87.830143,87.739714,11/30/22 14:40,87.830143,-7.535714,closed,102380.207143,RSI


In [68]:
# STOCH Strategy Run
STOCH_Strategy_etf5 = STOCH_Strategy(etf5, STARTING_BALANCE)
STOCH_run_etf5 = STOCH_Strategy_etf5.run()
STOCH_run_etf5['Strategy'] = 'Stochastic'
STOCH_run_etf5

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:21,100.76,SELL,250,107.760079,86.759841,9/22/22 9:41,86.759841,3500.039697,closed,103500.039697,Stochastic
1,9/22/22 9:41,86.69,BUY,250,72.689841,100.690159,None,NaN,NaN,open,NaN,Stochastic


In [69]:
# MFI Strategy Run

mfi_strategy_etf5 = MFI_Strategy(etf5, STARTING_BALANCE)
MFI_run_etf5 = mfi_strategy_etf5.run()
MFI_run_etf5['Strategy'] = 'MFI'
MFI_run_etf5

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:15,100.28,BUY,250,86.279841,121.280238,9/23/22 9:30,86.279841,-3500.039697,closed,96499.960303,MFI
1,9/23/22 9:30,85.75,BUY,250,71.749841,106.750238,None,NaN,NaN,open,NaN,MFI


In [70]:
etf5_combined = pd.concat([MFI_run_etf5, RSI_run_etf5, macd_trade_etf5, BBresult_etf5], ignore_index=True, sort=True)
etf5_combined = etf5_combined.sort_values(by='open_datetime')

In [71]:
etf5_combined = etf5_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [72]:
etf5_combined['Ticker'] = 'VOX'

In [73]:
etf5_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
2926,RSI,10/10/22 10:30,10/10/22 10:29,SELL,10.289286,closed,VOX
2927,RSI,10/10/22 11:27,10/10/22 11:26,SELL,3.371429,closed,VOX
2928,RSI,10/10/22 11:29,10/10/22 11:27,SELL,-1.685714,closed,VOX
2929,RSI,10/10/22 11:33,10/10/22 11:29,SELL,3.371429,closed,VOX
2930,RSI,10/10/22 11:35,10/10/22 11:33,SELL,-1.685714,closed,VOX
...,...,...,...,...,...,...,...
2386,RSI,9/9/22 14:40,9/9/22 14:39,SELL,2.964286,closed,VOX
2387,RSI,9/9/22 14:44,9/9/22 14:40,SELL,2.964286,closed,VOX
2388,RSI,9/9/22 15:13,9/9/22 15:07,BUY,4.250000,closed,VOX
2389,RSI,9/9/22 15:20,9/9/22 15:13,BUY,-4.250000,closed,VOX


In [169]:
etf5_combined.to_csv(r'/Users/krutarth/Downloads/vox_analysis.csv')

In [74]:
etf6 = data.iloc[:,40:48]

In [75]:
etf6 = etf6.rename(columns={'Dates.5': 'time', 'Open.5': 'Open', 'Close.5' : 'Close', 'Low.5' : 'Low', 
                            'Value.5' : 'Value', 'Volume.5' : 'Volume', 'Number Ticks.5' : 'Number Ticks', 'High.5':'High'})

In [76]:
etf6 = etf6.dropna()

In [77]:
# Generate Stichastic Oscillator 
etf6['slowk'], etf6['slowd'] = talib.STOCH(etf6['High'], etf6['Low'], etf6['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf6['MFI'] = talib.MFI(etf6['High'], etf6['Low'], etf6['Close'], etf6['Volume'], timeperiod=14)

# generate bollinger bands
etf6['BBupper'], etf6['BBmiddle'], etf6['BBlower'] = talib.BBANDS(etf6['Close'], matype=MA_Type.T3)
etf6['sd']=etf6['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf6['BBsignal']=np.vectorize(trade_signal)(etf6['Close'], etf6['BBlower'],etf6['BBupper'])

# Generate MACD trading signals
etf6['macd'], etf6['macdsignal'], etf6['macdhist'] = talib.MACD(etf6['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf6['prev_macd']=etf6['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf6['macd_trade_signal']=np.vectorize(trade_signal)(etf6['prev_macd'], etf6['macd'],etf6['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf6['gain'] = (etf6['Close']- etf6['Open']).apply(lambda x: x if x > 0 else 0)
etf6['loss'] = (etf6['Close']- etf6['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf6['ema_gain'] = etf6['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf6['ema_loss'] = etf6['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf6['rs'] = etf6['ema_gain'] / etf6['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf6['rsi_14'] = 100 - (100 / (etf6['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf6['range'] = etf6['High'] - etf6['Low']

# calculating the average value of ranges
etf6['atr_14'] = etf6['range'].rolling(atr_period).mean()

etf6=etf6.dropna()

In [78]:
# run the backtest
bollinger_strategy_etf6 = BBStrategy(etf6, STARTING_BALANCE, 250) 
BBresult_etf6 = bollinger_strategy_etf6.run().dropna()
BBresult_etf6['Strategy'] = 'Bollinger Bands'
BBresult_etf6

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:23,81.0350,BUY,250,77.223322,85.609014,5/26/22 12:52,85.609014,1143.503515,closed,101143.503515,Bollinger Bands
1,5/26/22 12:54,85.6700,SELL,250,89.481678,81.095986,6/10/22 9:34,81.095986,1143.503515,closed,102287.007031,Bollinger Bands
2,6/10/22 9:36,81.0950,SELL,250,84.906678,76.520986,6/14/22 14:48,76.520986,1143.503515,closed,103430.510546,Bollinger Bands
3,6/14/22 15:12,76.4000,BUY,250,72.588322,80.974014,7/22/22 9:30,80.974014,1143.503515,closed,104574.014061,Bollinger Bands
4,7/22/22 9:45,81.1800,BUY,250,77.368322,85.754014,8/11/22 9:30,85.754014,1143.503515,closed,105717.517576,Bollinger Bands
5,8/11/22 9:40,86.3324,BUY,250,82.520722,90.906414,8/29/22 9:30,82.520722,-952.919596,closed,104764.597980,Bollinger Bands
6,8/29/22 9:42,82.4699,SELL,250,86.281578,77.895886,9/23/22 9:30,77.895886,1143.503515,closed,105908.101496,Bollinger Bands
7,9/23/22 9:34,77.2000,BUY,250,73.388322,81.774014,10/13/22 9:30,73.388322,-952.919596,closed,104955.181900,Bollinger Bands
8,10/13/22 9:36,73.0800,BUY,250,69.268322,77.654014,10/13/22 13:32,77.654014,1143.503515,closed,106098.685415,Bollinger Bands
9,10/13/22 13:36,77.5900,BUY,250,73.778322,82.164014,10/27/22 9:31,82.164014,1143.503515,closed,107242.188930,Bollinger Bands


In [79]:
macd_trd_strategy_etf6=macd_Strategy(etf6, STARTING_BALANCE, 250)
macd_trade_etf6 = macd_trd_strategy_etf6.run().dropna()
macd_trade_etf6['Strategy'] = 'MACD'
macd_trade_etf6

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:06,80.8740,BUY,250,77.062322,85.448014,5/26/22 10:13,85.448014,1143.503515,closed,101143.503515,MACD
1,5/26/22 10:16,85.4900,BUY,250,81.678322,90.064014,6/10/22 9:30,81.678322,-952.919596,closed,100190.583919,MACD
2,6/10/22 9:37,81.3410,BUY,250,77.529322,85.915014,6/13/22 10:31,77.529322,-952.919596,closed,99237.664323,MACD
3,6/13/22 10:32,77.4400,SELL,250,81.251678,72.865986,7/22/22 9:32,81.251678,-952.919596,closed,98284.744727,MACD
4,7/22/22 9:34,81.2050,BUY,250,77.393322,85.779014,8/11/22 9:30,85.779014,1143.503515,closed,99428.248242,MACD
5,8/11/22 9:31,86.3600,BUY,250,82.548322,90.934014,8/29/22 9:30,82.548322,-952.919596,closed,98475.328646,MACD
6,8/29/22 9:37,82.4900,BUY,250,78.678322,87.064014,9/22/22 9:49,78.678322,-952.919596,closed,97522.409050,MACD
7,9/22/22 9:50,78.7500,BUY,250,74.938322,83.324014,9/27/22 12:10,74.938322,-952.919596,closed,96569.489454,MACD
8,9/27/22 12:14,74.8400,BUY,250,71.028322,79.414014,10/4/22 12:28,79.414014,1143.503515,closed,97712.992969,MACD
9,10/4/22 12:29,79.4499,BUY,250,75.638222,84.023914,10/7/22 15:27,75.638222,-952.919596,closed,96760.073373,MACD


In [80]:
# creating an instance of Strategy class
rsi_strategy_etf6 = RSI_Strategy(etf6, STARTING_BALANCE)

# running the backtest
RSI_run_etf6 = rsi_strategy_etf6.run()
RSI_run_etf6['Strategy'] = 'RSI'
RSI_run_etf6

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:29,80.470,SELL,250,80.504829,80.400343,5/19/22 11:30,80.504829,-8.707143,closed,99991.292857,RSI
1,5/19/22 11:34,80.790,SELL,250,80.828157,80.713686,5/19/22 11:36,80.828157,-9.539286,closed,99981.753571,RSI
2,5/19/22 11:36,80.840,SELL,250,80.872207,80.775586,5/19/22 11:38,80.872207,-8.051786,closed,99973.701786,RSI
3,5/19/22 11:41,81.235,SELL,250,81.274357,81.156286,5/19/22 11:42,81.274357,-9.839286,closed,99963.862500,RSI
4,5/19/22 11:42,81.285,SELL,250,81.326136,81.202729,5/19/22 11:45,81.202729,20.567857,closed,99984.430357,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
2407,11/30/22 15:15,87.935,SELL,250,87.964679,87.875643,11/30/22 15:16,87.964679,-7.419643,closed,100651.608929,RSI
2408,11/30/22 15:16,88.020,SELL,250,88.049679,87.960643,11/30/22 15:17,88.049679,-7.419643,closed,100644.189286,RSI
2409,11/30/22 15:17,88.080,SELL,250,88.109679,88.020643,11/30/22 15:18,88.020643,14.839286,closed,100659.028571,RSI
2410,11/30/22 15:34,88.300,SELL,250,88.325957,88.248086,11/30/22 15:35,88.325957,-6.489286,closed,100652.539286,RSI


In [81]:
# STOCH Strategy Run
STOCH_Strategy_etf6 = STOCH_Strategy(etf6, STARTING_BALANCE)
STOCH_run_etf6 = STOCH_Strategy_etf6.run()
STOCH_run_etf6['Strategy'] = 'Stochastic'
STOCH_run_etf6

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:25,81.04,BUY,250,73.416643,88.663357,10/13/22 9:30,73.416643,-1905.839192,closed,98094.160808,Stochastic
1,10/13/22 9:30,73.27,BUY,250,65.646643,80.893357,10/25/22 15:14,80.893357,1905.839192,closed,100000.000000,Stochastic
2,10/25/22 15:14,80.90,SELL,250,84.711678,73.276643,11/2/22 14:06,84.711678,-952.919596,closed,99047.080404,Stochastic
3,11/2/22 14:06,84.80,SELL,250,88.611678,77.176643,None,NaN,NaN,open,NaN,Stochastic


In [82]:
# MFI Strategy Run

mfi_strategy_etf6 = MFI_Strategy(etf6, STARTING_BALANCE)
MFI_run_etf6 = mfi_strategy_etf6.run()
MFI_run_etf6['Strategy'] = 'MFI'
MFI_run_etf6

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:10,80.8688,BUY,250,73.245443,92.303835,10/13/22 9:31,73.245443,-1905.839192,closed,98094.160808,MFI
1,10/13/22 9:31,73.0400,BUY,250,65.416643,84.475035,11/2/22 14:01,84.475035,2858.758788,closed,100952.919596,MFI
2,11/2/22 14:03,84.6300,SELL,250,88.441678,77.006643,11/11/22 10:26,88.441678,-952.919596,closed,100000.000000,MFI
3,11/11/22 10:34,88.0900,BUY,250,80.466643,99.525035,None,NaN,NaN,open,NaN,MFI


In [83]:
etf6_combined = pd.concat([MFI_run_etf6, RSI_run_etf6, macd_trade_etf6, BBresult_etf6], ignore_index=True, sort=True)
etf6_combined = etf6_combined.sort_values(by='open_datetime')

In [84]:
etf6_combined = etf6_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [85]:
etf6_combined['Ticker'] = 'VFH'

In [86]:
etf6_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
1784,RSI,10/10/22 11:45,10/10/22 11:43,SELL,-3.001786,closed,VFH
1785,RSI,10/10/22 11:47,10/10/22 11:45,SELL,-2.669643,closed,VFH
1786,RSI,10/10/22 12:13,10/10/22 12:08,BUY,-4.539286,closed,VFH
1787,RSI,10/10/22 12:15,10/10/22 12:13,BUY,3.289286,closed,VFH
1788,RSI,10/10/22 12:16,10/10/22 12:15,BUY,3.742857,closed,VFH
...,...,...,...,...,...,...,...
1436,RSI,9/9/22 14:35,9/9/22 14:33,BUY,-2.142857,closed,VFH
1437,RSI,9/9/22 14:38,9/9/22 14:35,BUY,2.142857,closed,VFH
1438,RSI,9/9/22 14:51,9/9/22 14:47,BUY,-7.375000,closed,VFH
1439,RSI,9/9/22 15:12,9/9/22 15:11,SELL,-1.532143,closed,VFH


In [170]:
etf6_combined.to_csv(r'/Users/krutarth/Downloads/vfh_analysis.csv')

In [87]:
etf7 = data.iloc[:,48:56]

In [88]:
etf7 = etf7.rename(columns={'Dates.6': 'time', 'Open.6': 'Open', 'Close.6' : 'Close', 'Low.6' : 'Low', 
                            'Value.6' : 'Value', 'Volume.6' : 'Volume', 'Number Ticks.6' : 'Number Ticks', 'High.6':'High'})

In [89]:
etf7 = etf7.dropna()

In [90]:
# Generate Stichastic Oscillator 
etf7['slowk'], etf7['slowd'] = talib.STOCH(etf7['High'], etf7['Low'], etf7['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf7['MFI'] = talib.MFI(etf7['High'], etf7['Low'], etf7['Close'], etf7['Volume'], timeperiod=14)

# generate bollinger bands
etf7['BBupper'], etf7['BBmiddle'], etf7['BBlower'] = talib.BBANDS(etf7['Close'], matype=MA_Type.T3)
etf7['sd']=etf7['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf7['BBsignal']=np.vectorize(trade_signal)(etf7['Close'], etf7['BBlower'],etf7['BBupper'])

# Generate MACD trading signals
etf7['macd'], etf7['macdsignal'], etf7['macdhist'] = talib.MACD(etf7['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf7['prev_macd']=etf7['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf7['macd_trade_signal']=np.vectorize(trade_signal)(etf7['prev_macd'], etf7['macd'],etf7['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf7['gain'] = (etf7['Close']- etf7['Open']).apply(lambda x: x if x > 0 else 0)
etf7['loss'] = (etf7['Close']- etf7['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf7['ema_gain'] = etf7['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf7['ema_loss'] = etf7['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf7['rs'] = etf7['ema_gain'] / etf7['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf7['rsi_14'] = 100 - (100 / (etf7['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf7['range'] = etf7['High'] - etf7['Low']

# calculating the average value of ranges
etf7['atr_14'] = etf7['range'].rolling(atr_period).mean()

etf7=etf7.dropna()

In [91]:
# run the backtest
bollinger_strategy_etf7 = BBStrategy(etf7, STARTING_BALANCE, 250) 
BBresult_etf7 = bollinger_strategy_etf7.run().dropna()
BBresult_etf7['Strategy'] = 'Bollinger Bands'
BBresult_etf7

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:00,168.880,SELL,250,178.086005,157.832794,5/27/22 9:53,178.086005,-2301.501245,closed,97698.498755,Bollinger Bands
1,5/27/22 9:59,178.084,BUY,250,168.877995,189.131206,6/13/22 9:30,168.877995,-2301.501245,closed,95396.997511,Bollinger Bands
2,6/13/22 9:37,167.160,SELL,250,176.366005,156.112794,7/28/22 13:34,176.366005,-2301.501245,closed,93095.496266,Bollinger Bands
3,7/28/22 13:41,176.590,SELL,250,185.796005,165.542794,8/10/22 11:01,185.796005,-2301.501245,closed,90793.995022,Bollinger Bands
4,8/10/22 12:30,185.550,BUY,250,176.343995,196.597206,8/31/22 13:43,176.343995,-2301.501245,closed,88492.493777,Bollinger Bands
5,8/31/22 14:01,176.290,SELL,250,185.496005,165.242794,9/22/22 9:45,165.242794,2761.801494,closed,91254.295270,Bollinger Bands
6,9/22/22 10:11,165.270,SELL,250,174.476005,154.222794,10/26/22 11:04,174.476005,-2301.501245,closed,88952.794026,Bollinger Bands
7,10/26/22 11:27,174.870,BUY,250,165.663995,185.917206,11/10/22 11:00,185.917206,2761.801494,closed,91714.595519,Bollinger Bands


In [92]:
macd_trd_strategy_etf7=macd_Strategy(etf7, STARTING_BALANCE, 250)
macd_trade_etf7 = macd_trd_strategy_etf7.run().dropna()
macd_trade_etf7['Strategy'] = 'MACD'
macd_trade_etf7

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:00,168.880,BUY,250,159.673995,179.927206,6/2/22 15:25,179.927206,2761.801494,closed,102761.801494,MACD
1,6/2/22 15:26,179.855,SELL,250,189.061005,168.807794,6/13/22 9:30,168.807794,2761.801494,closed,105523.602987,MACD
2,6/13/22 9:42,166.530,BUY,250,157.323995,177.577206,7/29/22 9:40,177.577206,2761.801494,closed,108285.404481,MACD
3,7/29/22 9:41,177.604,BUY,250,168.397995,188.651206,8/12/22 14:23,188.651206,2761.801494,closed,111047.205974,MACD
4,8/12/22 14:32,188.910,BUY,250,179.703995,199.957206,8/29/22 9:30,179.703995,-2301.501245,closed,108745.704730,MACD
5,8/29/22 9:52,179.371,BUY,250,170.164995,190.418206,9/16/22 9:30,170.164995,-2301.501245,closed,106444.203485,MACD
6,9/16/22 9:48,168.960,BUY,250,159.753995,180.007206,9/23/22 14:13,159.753995,-2301.501245,closed,104142.702240,MACD
7,9/23/22 14:24,159.750,BUY,250,150.543995,170.797206,10/25/22 9:48,170.797206,2761.801494,closed,106904.503734,MACD
8,10/25/22 9:49,171.240,BUY,250,162.033995,182.287206,11/8/22 11:16,182.287206,2761.801494,closed,109666.305227,MACD


In [93]:
# creating an instance of Strategy class
rsi_strategy_etf7 = RSI_Strategy(etf7, STARTING_BALANCE)

# running the backtest
RSI_run_etf7 = rsi_strategy_etf7.run()
RSI_run_etf7['Strategy'] = 'RSI'
RSI_run_etf7

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:40,170.600,SELL,250,170.659857,170.480286,5/19/22 11:41,170.659857,-14.964286,closed,99985.035714,RSI
1,5/19/22 11:41,170.760,SELL,250,170.825143,170.629714,5/19/22 11:42,170.825143,-16.285714,closed,99968.750000,RSI
2,5/19/22 11:42,170.880,SELL,250,170.945143,170.749714,5/19/22 11:45,170.749714,32.571429,closed,100001.321429,RSI
3,5/19/22 12:19,170.530,BUY,250,170.462286,170.597714,5/19/22 12:21,170.462286,-16.928571,closed,99984.392857,RSI
4,5/19/22 12:21,170.080,BUY,250,170.012286,170.147714,5/19/22 12:23,170.147714,16.928571,closed,100001.321429,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
3778,11/30/22 15:52,190.470,SELL,250,190.503143,190.403714,11/30/22 15:53,190.503143,-8.285714,closed,102933.196429,RSI
3779,11/30/22 15:53,190.550,SELL,250,190.578571,190.492857,11/30/22 15:55,190.578571,-7.142857,closed,102926.053571,RSI
3780,11/30/22 15:55,190.590,SELL,250,190.611429,190.547143,11/30/22 15:58,190.547143,10.714286,closed,102936.767857,RSI
3781,11/30/22 15:58,190.385,SELL,250,190.406429,190.342143,11/30/22 15:59,190.406429,-5.357143,closed,102931.410714,RSI


In [94]:
# STOCH Strategy Run
STOCH_Strategy_etf7 = STOCH_Strategy(etf7, STARTING_BALANCE)
STOCH_run_etf7 = STOCH_Strategy_etf7.run()
STOCH_run_etf7['Strategy'] = 'Stochastic'
STOCH_run_etf7

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:44,168.710,BUY,250,150.297990,187.12201,8/11/22 9:33,187.12201,4603.002489,closed,104603.002489,Stochastic
1,8/11/22 9:33,187.231,SELL,250,196.437005,168.81899,9/16/22 9:55,168.81899,4603.002489,closed,109206.004978,Stochastic
2,9/16/22 10:00,168.800,BUY,250,150.387990,187.21201,11/10/22 15:53,187.21201,4603.002489,closed,113809.007468,Stochastic
3,11/10/22 15:53,187.240,SELL,250,196.446005,168.82799,None,NaN,NaN,open,NaN,Stochastic


In [95]:
# MFI Strategy Run

mfi_strategy_etf7 = MFI_Strategy(etf7, STARTING_BALANCE)
MFI_run_etf7 = mfi_strategy_etf7.run()
MFI_run_etf7['Strategy'] = 'MFI'
MFI_run_etf7

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:13,169.045,SELL,250,178.251005,150.632990,5/27/22 9:53,178.251005,-2301.501245,closed,97698.498755,MFI
1,5/27/22 9:53,178.335,SELL,250,187.541005,159.922990,6/16/22 10:52,159.922990,4603.002489,closed,102301.501245,MFI
2,6/16/22 11:05,159.980,BUY,250,141.567990,187.598015,8/11/22 9:57,187.598015,6904.503734,closed,109206.004978,MFI
3,8/11/22 10:05,187.879,SELL,250,197.085005,169.466990,9/16/22 9:30,169.466990,4603.002489,closed,113809.007468,MFI
4,9/16/22 9:46,169.020,BUY,250,150.607990,196.638015,None,NaN,NaN,open,NaN,MFI


In [96]:
etf7_combined = pd.concat([MFI_run_etf7, RSI_run_etf7, macd_trade_etf7, BBresult_etf7], ignore_index=True, sort=True)
etf7_combined = etf7_combined.sort_values(by='open_datetime')

In [97]:
etf7_combined = etf7_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [98]:
etf7_combined['Ticker'] = 'VIS'

In [99]:
etf7_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
2596,RSI,10/10/22 10:05,10/10/22 10:03,SELL,-14.803571,closed,VIS
2597,RSI,10/10/22 10:06,10/10/22 10:05,SELL,-16.964286,closed,VIS
2598,RSI,10/10/22 12:57,10/10/22 12:43,BUY,-4.892857,closed,VIS
2599,RSI,10/10/22 13:10,10/10/22 12:57,BUY,-4.892857,closed,VIS
2600,RSI,10/10/22 13:11,10/10/22 13:10,BUY,4.892857,closed,VIS
...,...,...,...,...,...,...,...
2112,RSI,9/9/22 15:29,9/9/22 15:11,SELL,-1.232143,closed,VIS
2113,RSI,9/9/22 15:40,9/9/22 15:29,SELL,2.607143,closed,VIS
2114,RSI,9/9/22 15:46,9/9/22 15:40,SELL,-1.303571,closed,VIS
2115,RSI,9/9/22 15:57,9/9/22 15:46,SELL,2.607143,closed,VIS


In [171]:
etf7_combined.to_csv(r'/Users/krutarth/Downloads/vis_analysis.csv')

In [100]:
etf8 = data.iloc[:,56:64]

In [101]:
etf8 = etf8.rename(columns={'Dates.7': 'time', 'Open.7': 'Open', 'Close.7' : 'Close', 'Low.7' : 'Low', 
                            'Value.7' : 'Value', 'Volume.7' : 'Volume', 'Number Ticks.7' : 'Number Ticks', 'High.7':'High'})

In [102]:
etf8 = etf8.dropna()

In [103]:
# Generate Stichastic Oscillator 
etf8['slowk'], etf8['slowd'] = talib.STOCH(etf8['High'], etf8['Low'], etf8['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf8['MFI'] = talib.MFI(etf8['High'], etf8['Low'], etf8['Close'], etf8['Volume'], timeperiod=14)

# generate bollinger bands
etf8['BBupper'], etf8['BBmiddle'], etf8['BBlower'] = talib.BBANDS(etf8['Close'], matype=MA_Type.T3)
etf8['sd']=etf8['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf8['BBsignal']=np.vectorize(trade_signal)(etf8['Close'], etf8['BBlower'],etf8['BBupper'])

# Generate MACD trading signals
etf8['macd'], etf8['macdsignal'], etf8['macdhist'] = talib.MACD(etf8['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf8['prev_macd']=etf8['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf8['macd_trade_signal']=np.vectorize(trade_signal)(etf8['prev_macd'], etf8['macd'],etf8['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf8['gain'] = (etf8['Close']- etf8['Open']).apply(lambda x: x if x > 0 else 0)
etf8['loss'] = (etf8['Close']- etf8['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf8['ema_gain'] = etf8['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf8['ema_loss'] = etf8['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf8['rs'] = etf8['ema_gain'] / etf8['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf8['rsi_14'] = 100 - (100 / (etf8['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf8['range'] = etf8['High'] - etf8['Low']

# calculating the average value of ranges
etf8['atr_14'] = etf8['range'].rolling(atr_period).mean()

etf8=etf8.dropna()

In [104]:
# run the backtest
bollinger_strategy_etf8 = BBStrategy(etf8, STARTING_BALANCE, 250) 
BBresult_etf8 = bollinger_strategy_etf8.run().dropna()
BBresult_etf8['Strategy'] = 'Bollinger Bands'
BBresult_etf8

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:16,182.140,SELL,250,188.748942,174.20927,5/25/22 15:03,188.748942,-1652.235421,closed,98347.764579,Bollinger Bands
1,5/25/22 15:14,188.600,BUY,250,181.991058,196.53073,6/13/22 9:41,181.991058,-1652.235421,closed,96695.529157,Bollinger Bands
2,6/13/22 9:51,182.410,SELL,250,189.018942,174.47927,7/25/22 10:14,189.018942,-1652.235421,closed,95043.293736,Bollinger Bands
3,7/25/22 10:26,188.880,BUY,250,182.271058,196.81073,8/16/22 9:30,196.810730,1982.682506,closed,97025.976241,Bollinger Bands
4,8/16/22 9:35,197.670,SELL,250,204.278942,189.73927,8/30/22 10:49,189.739270,1982.682506,closed,99008.658747,Bollinger Bands
5,8/30/22 10:55,189.845,SELL,250,196.453942,181.91427,9/23/22 9:30,181.914270,1982.682506,closed,100991.341253,Bollinger Bands
6,9/23/22 9:35,180.700,BUY,250,174.091058,188.63073,9/29/22 13:51,174.091058,-1652.235421,closed,99339.105831,Bollinger Bands
7,9/29/22 13:58,173.875,BUY,250,167.266058,181.80573,10/25/22 9:56,181.805730,1982.682506,closed,101321.788337,Bollinger Bands
8,10/25/22 10:09,181.340,BUY,250,174.731058,189.27073,10/28/22 15:03,189.270730,1982.682506,closed,103304.470843,Bollinger Bands
9,10/31/22 9:30,188.010,BUY,250,181.401058,195.94073,11/22/22 9:30,195.940730,1982.682506,closed,105287.153349,Bollinger Bands


In [105]:
macd_trd_strategy_etf8=macd_Strategy(etf8, STARTING_BALANCE, 250)
macd_trade_etf8 = macd_trd_strategy_etf8.run().dropna()
macd_trade_etf8['Strategy'] = 'MACD'
macd_trade_etf8

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:05,181.63,BUY,250,175.021058,189.56073,5/26/22 9:30,189.560730,1982.682506,closed,101982.682506,MACD
1,5/26/22 9:31,189.81,BUY,250,183.201058,197.74073,6/10/22 9:34,183.201058,-1652.235421,closed,100330.447084,MACD
2,6/10/22 9:41,183.18,BUY,250,176.571058,191.11073,6/16/22 9:36,176.571058,-1652.235421,closed,98678.211663,MACD
3,6/16/22 9:43,175.93,BUY,250,169.321058,183.86073,6/24/22 9:30,183.860730,1982.682506,closed,100660.894169,MACD
4,6/24/22 9:31,185.36,BUY,250,178.751058,193.29073,8/1/22 9:51,193.290730,1982.682506,closed,102643.576674,MACD
5,8/1/22 9:52,193.28,BUY,250,186.671058,201.21073,9/2/22 15:38,186.671058,-1652.235421,closed,100991.341253,MACD
6,9/2/22 15:45,186.50,BUY,250,179.891058,194.43073,9/23/22 11:05,179.891058,-1652.235421,closed,99339.105831,MACD
7,9/23/22 11:09,180.12,BUY,250,173.511058,188.05073,9/30/22 9:36,173.511058,-1652.235421,closed,97686.870410,MACD
8,9/30/22 9:43,174.00,BUY,250,167.391058,181.93073,10/25/22 9:56,181.930730,1982.682506,closed,99669.552916,MACD
9,10/25/22 9:58,182.01,BUY,250,175.401058,189.94073,11/10/22 9:44,189.940730,1982.682506,closed,101652.235421,MACD


In [106]:
# creating an instance of Strategy class
rsi_strategy_etf8 = RSI_Strategy(etf8, STARTING_BALANCE)

# running the backtest
RSI_run_etf8 = rsi_strategy_etf8.run()
RSI_run_etf8['Strategy'] = 'RSI'
RSI_run_etf8

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:09,180.875,SELL,250,180.931143,180.762714,5/19/22 11:11,180.931143,-14.035714,closed,99985.964286,RSI
1,5/19/22 11:11,181.119,SELL,250,181.172643,181.011714,5/19/22 11:13,181.011714,26.821429,closed,100012.785714,RSI
2,5/19/22 11:34,182.290,SELL,250,182.355714,182.158571,5/19/22 11:37,182.355714,-16.428571,closed,99996.357143,RSI
3,5/19/22 11:37,182.540,SELL,250,182.606429,182.407143,5/19/22 11:38,182.606429,-16.607143,closed,99979.750000,RSI
4,5/19/22 11:38,182.620,SELL,250,182.684286,182.491429,5/19/22 11:40,182.684286,-16.071429,closed,99963.678571,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
4071,11/30/22 15:19,198.710,BUY,250,198.682143,198.737857,11/30/22 15:22,198.737857,6.964286,closed,103260.017857,RSI
4072,11/30/22 15:22,198.775,BUY,250,198.747143,198.802857,11/30/22 15:25,198.802857,6.964286,closed,103266.982143,RSI
4073,11/30/22 15:40,199.100,SELL,250,199.118214,199.063571,11/30/22 15:41,199.063571,9.107143,closed,103276.089286,RSI
4074,11/30/22 15:41,199.060,SELL,250,199.077143,199.025714,11/30/22 15:42,199.025714,8.571429,closed,103284.660714,RSI


In [107]:
# STOCH Strategy Run
STOCH_Strategy_etf8 = STOCH_Strategy(etf8, STARTING_BALANCE)
STOCH_run_etf8 = STOCH_Strategy_etf8.run()
STOCH_run_etf8['Strategy'] = 'Stochastic'
STOCH_run_etf8

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:09,181.69,BUY,250,168.472117,194.907883,8/15/22 9:45,194.907883,3304.470843,closed,103304.470843,Stochastic
1,8/15/22 9:45,194.95,SELL,250,201.558942,181.732117,9/23/22 9:30,181.732117,3304.470843,closed,106608.941686,Stochastic
2,9/23/22 9:30,181.19,BUY,250,167.972117,194.407883,11/21/22 10:39,194.407883,3304.470843,closed,109913.412529,Stochastic
3,11/21/22 10:39,194.48,SELL,250,201.088942,181.262117,None,NaN,NaN,open,NaN,Stochastic


In [108]:
# MFI Strategy Run

mfi_strategy_etf8 = MFI_Strategy(etf8, STARTING_BALANCE)
MFI_run_etf8 = mfi_strategy_etf8.run()
MFI_run_etf8['Strategy'] = 'MFI'
MFI_run_etf8

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:36,181.12,BUY,250,167.902117,200.946825,None,None,None,open,NaN,MFI


In [109]:
etf8_combined = pd.concat([MFI_run_etf8, RSI_run_etf8, macd_trade_etf8, BBresult_etf8], ignore_index=True, sort=True)
etf8_combined = etf8_combined.sort_values(by='open_datetime')

In [110]:
etf8_combined = etf8_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [111]:
etf8_combined['Ticker'] = 'VDC'

In [112]:
etf8_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
2939,RSI,10/10/22 11:49,10/10/22 11:47,SELL,5.000000,closed,VDC
2940,RSI,10/10/22 11:50,10/10/22 11:49,SELL,-2.857143,closed,VDC
2941,RSI,10/10/22 12:00,10/10/22 11:50,SELL,11.785714,closed,VDC
2942,RSI,10/10/22 12:01,10/10/22 12:00,SELL,-6.339286,closed,VDC
2943,RSI,10/10/22 12:03,10/10/22 12:01,SELL,14.107143,closed,VDC
...,...,...,...,...,...,...,...
2456,RSI,9/9/22 15:54,9/9/22 15:52,BUY,2.500000,closed,VDC
2457,RSI,9/9/22 15:56,9/9/22 15:54,BUY,-2.500000,closed,VDC
2458,RSI,9/9/22 15:57,9/9/22 15:56,BUY,-2.500000,closed,VDC
2459,RSI,9/9/22 15:59,9/9/22 15:57,BUY,4.285714,closed,VDC


In [172]:
etf8_combined.to_csv(r'/Users/krutarth/Downloads/vdc_analysis.csv')

In [113]:
etf9 = data.iloc[:,64:72]

In [114]:
etf9 = etf9.rename(columns={'Dates.8': 'time', 'Open.8': 'Open', 'Close.8' : 'Close', 'Low.8' : 'Low', 
                            'Value.8' : 'Value', 'Volume.8' : 'Volume', 'Number Ticks.8' : 'Number Ticks', 'High.8':'High'})

In [115]:
etf9 = etf9.dropna()

In [116]:
# Generate Stichastic Oscillator 
etf9['slowk'], etf9['slowd'] = talib.STOCH(etf9['High'], etf9['Low'], etf9['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf9['MFI'] = talib.MFI(etf9['High'], etf9['Low'], etf9['Close'], etf9['Volume'], timeperiod=14)

# generate bollinger bands
etf9['BBupper'], etf9['BBmiddle'], etf9['BBlower'] = talib.BBANDS(etf9['Close'], matype=MA_Type.T3)
etf9['sd']=etf9['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf9['BBsignal']=np.vectorize(trade_signal)(etf9['Close'], etf9['BBlower'],etf9['BBupper'])

# Generate MACD trading signals
etf9['macd'], etf9['macdsignal'], etf9['macdhist'] = talib.MACD(etf9['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf9['prev_macd']=etf9['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf9['macd_trade_signal']=np.vectorize(trade_signal)(etf9['prev_macd'], etf9['macd'],etf9['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf9['gain'] = (etf9['Close']- etf9['Open']).apply(lambda x: x if x > 0 else 0)
etf9['loss'] = (etf9['Close']- etf9['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf9['ema_gain'] = etf9['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf9['ema_loss'] = etf9['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf9['rs'] = etf9['ema_gain'] / etf9['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf9['rsi_14'] = 100 - (100 / (etf9['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf9['range'] = etf9['High'] - etf9['Low']

# calculating the average value of ranges
etf9['atr_14'] = etf9['range'].rolling(atr_period).mean()

etf9=etf9.dropna()

In [117]:
# run the backtest
bollinger_strategy_etf9 = BBStrategy(etf9, STARTING_BALANCE, 250) 
BBresult_etf9 = bollinger_strategy_etf9.run().dropna()
BBresult_etf9['Strategy'] = 'Bollinger Bands'
BBresult_etf9

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:11,154.400,SELL,250,163.606488,143.352215,5/27/22 15:57,163.606488,-2301.621882,closed,97698.378118,Bollinger Bands
1,5/27/22 15:59,163.800,SELL,250,173.006488,152.752215,6/13/22 9:43,152.752215,2761.946258,closed,100460.324376,Bollinger Bands
2,6/13/22 9:47,152.770,SELL,250,161.976488,141.722215,6/17/22 10:24,141.722215,2761.946258,closed,103222.270634,Bollinger Bands
3,6/17/22 10:56,140.680,SELL,250,149.886488,129.632215,6/24/22 13:42,149.886488,-2301.621882,closed,100920.648753,Bollinger Bands
4,6/24/22 13:47,149.945,SELL,250,159.151488,138.897215,7/28/22 13:57,159.151488,-2301.621882,closed,98619.026871,Bollinger Bands
5,7/28/22 14:13,159.130,SELL,250,168.336488,148.082215,8/15/22 15:56,168.336488,-2301.621882,closed,96317.404990,Bollinger Bands
6,8/16/22 9:30,167.930,BUY,250,158.723512,178.977785,9/22/22 9:31,158.723512,-2301.621882,closed,94015.783108,Bollinger Bands
7,9/22/22 9:41,158.330,BUY,250,149.123512,169.377785,9/28/22 9:39,149.123512,-2301.621882,closed,91714.161226,Bollinger Bands
8,9/28/22 9:43,149.540,SELL,250,158.746488,138.492215,10/7/22 13:08,138.492215,2761.946258,closed,94476.107484,Bollinger Bands
9,10/7/22 13:13,138.530,SELL,250,147.736488,127.482215,11/2/22 14:01,147.736488,-2301.621882,closed,92174.485603,Bollinger Bands


In [118]:
macd_trd_strategy_etf9=macd_Strategy(etf9, STARTING_BALANCE, 250)
macd_trade_etf9 = macd_trd_strategy_etf9.run().dropna()
macd_trade_etf9['Strategy'] = 'MACD'
macd_trade_etf9

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:07,154.095,BUY,250,144.888512,165.142785,6/14/22 11:15,144.888512,-2301.621882,closed,97698.378118,MACD
1,6/14/22 11:18,144.890,BUY,250,135.683512,155.937785,7/1/22 15:13,155.937785,2761.946258,closed,100460.324376,MACD
2,7/1/22 15:14,155.960,BUY,250,146.753512,167.007785,8/15/22 9:30,167.007785,2761.946258,closed,103222.270634,MACD
3,8/15/22 9:31,166.715,SELL,250,175.921488,155.667215,9/23/22 12:08,155.667215,2761.946258,closed,105984.216892,MACD
4,9/23/22 12:14,155.530,BUY,250,146.323512,166.577785,9/29/22 12:22,146.323512,-2301.621882,closed,103682.595010,MACD
5,9/29/22 12:38,146.150,BUY,250,136.943512,157.197785,10/12/22 11:24,136.943512,-2301.621882,closed,101380.973129,MACD
6,10/12/22 11:28,136.870,BUY,250,127.663512,147.917785,11/2/22 14:01,147.917785,2761.946258,closed,104142.919387,MACD


In [119]:
# creating an instance of Strategy class
rsi_strategy_etf9 = RSI_Strategy(etf9, STARTING_BALANCE)

# running the backtest
RSI_run_etf9 = rsi_strategy_etf9.run()
RSI_run_etf9['Strategy'] = 'RSI'
RSI_run_etf9

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:43,154.620,BUY,250,154.544429,154.695571,5/19/22 10:44,154.695571,18.892857,closed,100018.892857,RSI
1,5/19/22 10:44,154.699,BUY,250,154.623429,154.774571,5/19/22 10:47,154.623429,-18.892857,closed,100000.000000,RSI
2,5/19/22 11:50,155.010,BUY,250,154.975714,155.044286,5/19/22 11:51,155.044286,8.571429,closed,100008.571429,RSI
3,5/19/22 11:55,154.860,BUY,250,154.831429,154.888571,5/19/22 11:56,154.831429,-7.142857,closed,100001.428571,RSI
4,5/19/22 11:56,154.817,BUY,250,154.783714,154.850286,5/19/22 11:57,154.783714,-8.321429,closed,99993.107143,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
3202,11/30/22 15:08,155.200,SELL,250,155.214571,155.170857,11/30/22 15:09,155.214571,-3.642857,closed,100434.660714,RSI
3203,11/30/22 15:09,155.220,SELL,250,155.230357,155.199286,11/30/22 15:12,155.199286,5.178571,closed,100439.839286,RSI
3204,11/30/22 15:12,155.160,SELL,250,155.170357,155.139286,11/30/22 15:13,155.170357,-2.589286,closed,100437.250000,RSI
3205,11/30/22 15:13,155.206,SELL,250,155.213214,155.191571,11/30/22 15:15,155.191571,3.607143,closed,100440.857143,RSI


In [120]:
# STOCH Strategy Run
STOCH_Strategy_etf9 = STOCH_Strategy(etf9, STARTING_BALANCE)
STOCH_run_etf9 = STOCH_Strategy_etf9.run()
STOCH_run_etf9['Strategy'] = 'Stochastic'
STOCH_run_etf9

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:07,154.095,BUY,250,135.682025,172.507975,10/12/22 12:37,135.682025,-4603.243763,closed,95396.756237,Stochastic
1,10/12/22 12:37,135.641,BUY,250,117.228025,154.053975,11/23/22 15:59,154.053975,4603.243763,closed,100000.000000,Stochastic
2,11/23/22 15:59,154.090,SELL,250,163.296488,135.677025,None,NaN,NaN,open,NaN,Stochastic


In [121]:
# MFI Strategy Run

mfi_strategy_etf9 = MFI_Strategy(etf9, STARTING_BALANCE)
MFI_run_etf9 = mfi_strategy_etf9.run()
MFI_run_etf9['Strategy'] = 'MFI'
MFI_run_etf9

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:25,155.48,SELL,250,164.686488,137.067025,8/11/22 9:52,164.686488,-2301.621882,closed,97698.378118,MFI
1,8/11/22 9:52,164.72,BUY,250,146.307025,192.339463,9/29/22 12:25,146.307025,-4603.243763,closed,93095.134355,MFI
2,9/29/22 12:30,146.13,BUY,250,127.717025,173.749463,None,NaN,NaN,open,NaN,MFI


In [122]:
etf9_combined = pd.concat([MFI_run_etf9, RSI_run_etf9, macd_trade_etf9, BBresult_etf9], ignore_index=True, sort=True)
etf9_combined = etf9_combined.sort_values(by='open_datetime')

In [123]:
etf9_combined = etf9_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [124]:
etf9_combined['Ticker'] = 'VPU'

In [125]:
etf9_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
2327,RSI,10/10/22 12:18,10/10/22 12:15,BUY,-3.250000,closed,VPU
2328,RSI,10/10/22 12:19,10/10/22 12:18,BUY,6.107143,closed,VPU
2329,RSI,10/10/22 12:35,10/10/22 12:33,BUY,7.035714,closed,VPU
2330,RSI,10/10/22 12:36,10/10/22 12:35,BUY,5.964286,closed,VPU
2331,RSI,10/10/22 12:38,10/10/22 12:36,BUY,5.428571,closed,VPU
...,...,...,...,...,...,...,...
1861,RSI,9/9/22 15:38,9/9/22 15:37,BUY,-5.250000,closed,VPU
1862,RSI,9/9/22 15:39,9/9/22 15:38,BUY,5.250000,closed,VPU
1863,RSI,9/9/22 15:40,9/9/22 15:39,BUY,5.250000,closed,VPU
1864,RSI,9/9/22 15:41,9/9/22 15:40,BUY,3.821429,closed,VPU


In [173]:
etf9_combined.to_csv(r'/Users/krutarth/Downloads/vpu_analysis.csv')

In [126]:
etf10 = data.iloc[:,72:80]

In [127]:
etf10 = etf10.rename(columns={'Dates.9': 'time', 'Open.9': 'Open', 'Close.9' : 'Close', 'Low.9' : 'Low', 
                            'Value.9' : 'Value', 'Volume.9' : 'Volume', 'Number Ticks.9' : 'Number Ticks', 'High.9':'High'})

In [128]:
etf10 = etf10.dropna()

In [129]:
# Generate Stichastic Oscillator 
etf10['slowk'], etf10['slowd'] = talib.STOCH(etf10['High'], etf10['Low'], etf10['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf10['MFI'] = talib.MFI(etf10['High'], etf10['Low'], etf10['Close'], etf10['Volume'], timeperiod=14)

# generate bollinger bands
etf10['BBupper'], etf10['BBmiddle'], etf10['BBlower'] = talib.BBANDS(etf10['Close'], matype=MA_Type.T3)
etf10['sd']=etf10['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf10['BBsignal']=np.vectorize(trade_signal)(etf10['Close'], etf10['BBlower'],etf10['BBupper'])

# Generate MACD trading signals
etf10['macd'], etf10['macdsignal'], etf10['macdhist'] = talib.MACD(etf10['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf10['prev_macd']=etf10['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf10['macd_trade_signal']=np.vectorize(trade_signal)(etf10['prev_macd'], etf10['macd'],etf10['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf10['gain'] = (etf10['Close']- etf10['Open']).apply(lambda x: x if x > 0 else 0)
etf10['loss'] = (etf10['Close']- etf10['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf10['ema_gain'] = etf10['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf10['ema_loss'] = etf10['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf10['rs'] = etf10['ema_gain'] / etf10['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf10['rsi_14'] = 100 - (100 / (etf10['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf10['range'] = etf10['High'] - etf10['Low']

# calculating the average value of ranges
etf10['atr_14'] = etf10['range'].rolling(atr_period).mean()

etf10=etf10.dropna()

In [130]:
# run the backtest
bollinger_strategy_etf10 = BBStrategy(etf10, STARTING_BALANCE, 250) 
BBresult_etf10 = bollinger_strategy_etf10.run().dropna()
BBresult_etf10['Strategy'] = 'Bollinger Bands'
BBresult_etf10

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:53,179.13,SELL,250,189.933471,166.165834,5/27/22 11:45,189.933471,-2700.867843,closed,97299.132157,Bollinger Bands
1,5/27/22 12:05,189.92,BUY,250,179.116529,202.884166,6/10/22 10:03,179.116529,-2700.867843,closed,94598.264315,Bollinger Bands
2,6/10/22 10:11,178.69,BUY,250,167.886529,191.654166,6/15/22 14:24,167.886529,-2700.867843,closed,91897.396472,Bollinger Bands
3,6/15/22 14:31,167.33,BUY,250,156.526529,180.294166,7/5/22 9:33,156.526529,-2700.867843,closed,89196.528629,Bollinger Bands
4,7/5/22 9:36,156.12,BUY,250,145.316529,169.084166,7/28/22 15:52,169.084166,3241.041411,closed,92437.570040,Bollinger Bands
5,7/28/22 16:14,169.01,BUY,250,158.206529,181.974166,9/16/22 9:59,158.206529,-2700.867843,closed,89736.702198,Bollinger Bands
6,9/16/22 10:06,159.14,SELL,250,169.943471,146.175834,11/8/22 10:35,169.943471,-2700.867843,closed,87035.834355,Bollinger Bands


In [131]:
macd_trd_strategy_etf10=macd_Strategy(etf10, STARTING_BALANCE, 250)
macd_trade_etf10 = macd_trd_strategy_etf10.run().dropna()
macd_trade_etf10['Strategy'] = 'MACD'
macd_trade_etf10

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:53,179.130,BUY,250,168.326529,192.094166,6/6/22 10:06,192.094166,3241.041411,closed,103241.041411,MACD
1,6/6/22 10:07,192.130,SELL,250,202.933471,179.165834,6/10/22 10:03,179.165834,3241.041411,closed,106482.082822,MACD
2,6/10/22 10:06,179.392,BUY,250,168.588529,192.356166,6/14/22 15:12,168.588529,-2700.867843,closed,103781.214980,MACD
3,6/14/22 15:13,168.370,SELL,250,179.173471,155.405834,7/5/22 9:52,155.405834,3241.041411,closed,107022.256391,MACD
4,7/5/22 9:53,155.270,BUY,250,144.466529,168.234166,7/28/22 9:41,168.234166,3241.041411,closed,110263.297802,MACD
5,7/28/22 9:42,168.266,BUY,250,157.462529,181.230166,9/21/22 14:14,157.462529,-2700.867843,closed,107562.429960,MACD
6,9/21/22 14:24,157.530,BUY,250,146.726529,170.494166,9/29/22 10:35,146.726529,-2700.867843,closed,104861.562117,MACD
7,9/29/22 10:40,147.190,BUY,250,136.386529,160.154166,10/24/22 9:45,160.154166,3241.041411,closed,108102.603528,MACD
8,10/24/22 9:47,160.088,BUY,250,149.284529,173.052166,11/10/22 10:07,173.052166,3241.041411,closed,111343.644939,MACD


In [132]:
# creating an instance of Strategy class
rsi_strategy_etf10 = RSI_Strategy(etf10, STARTING_BALANCE)

# running the backtest
RSI_run_etf10 = rsi_strategy_etf10.run()
RSI_run_etf10['Strategy'] = 'RSI'
RSI_run_etf10

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 12:34,180.35,SELL,250,180.367857,180.314286,5/19/22 12:37,180.314286,8.928571,closed,100008.928571,RSI
1,5/19/22 12:37,180.08,SELL,250,180.088571,180.062857,5/19/22 12:39,180.062857,4.285714,closed,100013.214286,RSI
2,5/19/22 12:39,179.83,SELL,250,179.838571,179.812857,5/19/22 12:46,179.812857,4.285714,closed,100017.500000,RSI
3,5/19/22 12:46,179.50,SELL,250,179.508571,179.482857,5/19/22 12:47,179.508571,-2.142857,closed,100015.357143,RSI
4,5/19/22 12:47,179.57,SELL,250,179.578571,179.552857,5/19/22 12:48,179.578571,-2.142857,closed,100013.214286,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
3517,11/30/22 15:53,180.47,BUY,250,180.470000,180.470000,11/30/22 15:55,180.470000,0.000000,closed,101795.910714,RSI
3518,11/30/22 15:55,180.55,BUY,250,180.550000,180.550000,11/30/22 15:56,180.550000,0.000000,closed,101795.910714,RSI
3519,11/30/22 15:56,180.59,BUY,250,180.590000,180.590000,11/30/22 15:58,180.590000,0.000000,closed,101795.910714,RSI
3520,11/30/22 15:58,180.64,BUY,250,180.640000,180.640000,11/30/22 15:59,180.640000,0.000000,closed,101795.910714,RSI


In [133]:
# STOCH Strategy Run
STOCH_Strategy_etf10 = STOCH_Strategy(etf10, STARTING_BALANCE)
STOCH_run_etf10 = STOCH_Strategy_etf10.run()
STOCH_run_etf10['Strategy'] = 'Stochastic'
STOCH_run_etf10

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:41,178.81,BUY,250,157.203057,200.416943,7/1/22 10:40,157.203057,-5401.735685,closed,94598.264315,Stochastic
1,7/1/22 10:40,157.05,BUY,250,135.443057,178.656943,8/16/22 9:37,178.656943,5401.735685,closed,100000.000000,Stochastic
2,8/16/22 9:38,178.85,SELL,250,189.653471,157.243057,9/21/22 14:14,157.243057,5401.735685,closed,105401.735685,Stochastic
3,9/21/22 14:14,157.02,BUY,250,135.413057,178.626943,11/11/22 10:14,178.626943,5401.735685,closed,110803.471371,Stochastic
4,11/11/22 10:19,178.96,SELL,250,189.763471,157.353057,None,NaN,NaN,open,NaN,Stochastic


In [134]:
# MFI Strategy Run

mfi_strategy_etf10 = MFI_Strategy(etf10, STARTING_BALANCE)
MFI_run_etf10 = mfi_strategy_etf10.run()
MFI_run_etf10['Strategy'] = 'MFI'
MFI_run_etf10

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:32,179.97,SELL,250,190.773471,158.363057,5/27/22 15:27,190.773471,-2700.867843,closed,97299.132157,MFI
1,5/27/22 15:27,190.81,SELL,250,201.613471,169.203057,6/14/22 12:38,169.203057,5401.735685,closed,102700.867843,MFI
2,6/14/22 12:59,169.51,SELL,250,180.313471,147.903057,8/16/22 14:20,180.313471,-2700.867843,closed,100000.000000,MFI
3,8/16/22 14:20,180.41,SELL,250,191.213471,158.803057,9/16/22 9:33,158.803057,5401.735685,closed,105401.735685,MFI
4,9/16/22 9:33,158.77,BUY,250,137.163057,191.180414,None,NaN,NaN,open,NaN,MFI


In [135]:
etf10_combined = pd.concat([MFI_run_etf10, RSI_run_etf10, macd_trade_etf10, BBresult_etf10], ignore_index=True, sort=True)
etf10_combined = etf10_combined.sort_values(by='open_datetime')

In [136]:
etf10_combined = etf10_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [137]:
etf10_combined['Ticker'] = 'VAW'

In [138]:
etf10_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
2565,RSI,10/10/22 12:09,10/10/22 12:08,BUY,11.500000,closed,VAW
2566,RSI,10/10/22 12:12,10/10/22 12:09,BUY,-11.500000,closed,VAW
2567,RSI,10/10/22 12:15,10/10/22 12:12,BUY,9.714286,closed,VAW
2568,RSI,10/10/22 12:18,10/10/22 12:15,BUY,-9.714286,closed,VAW
2569,RSI,10/10/22 12:24,10/10/22 12:18,BUY,-10.071429,closed,VAW
...,...,...,...,...,...,...,...
2117,RSI,9/9/22 13:36,9/9/22 13:32,BUY,-2.500000,closed,VAW
2118,RSI,9/9/22 13:47,9/9/22 13:36,BUY,-1.071429,closed,VAW
2119,RSI,9/9/22 13:56,9/9/22 13:47,BUY,1.392857,closed,VAW
2120,RSI,9/9/22 13:57,9/9/22 13:56,BUY,-1.392857,closed,VAW


In [174]:
etf10_combined.to_csv(r'/Users/krutarth/Downloads/vaw_analysis.csv')

In [139]:
etf11 = data.iloc[:,80:88]

In [140]:
etf11 = etf11.rename(columns={'Dates.10': 'time', 'Open.10': 'Open', 'Close.10' : 'Close', 'Low.10' : 'Low', 
                            'Value.10' : 'Value', 'Volume.10' : 'Volume', 'Number Ticks.10' : 'Number Ticks', 'High.10':'High'})

In [141]:
etf11 = etf11.dropna()

In [142]:
# Generate Stichastic Oscillator 
etf11['slowk'], etf11['slowd'] = talib.STOCH(etf11['High'], etf11['Low'], etf11['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf11['MFI'] = talib.MFI(etf11['High'], etf11['Low'], etf11['Close'], etf11['Volume'], timeperiod=14)

# generate bollinger bands
etf11['BBupper'], etf11['BBmiddle'], etf11['BBlower'] = talib.BBANDS(etf11['Close'], matype=MA_Type.T3)
etf11['sd']=etf11['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf11['BBsignal']=np.vectorize(trade_signal)(etf11['Close'], etf11['BBlower'],etf11['BBupper'])

# Generate MACD trading signals
etf11['macd'], etf11['macdsignal'], etf11['macdhist'] = talib.MACD(etf11['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf11['prev_macd']=etf11['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf11['macd_trade_signal']=np.vectorize(trade_signal)(etf11['prev_macd'], etf11['macd'],etf11['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf11['gain'] = (etf11['Close']- etf11['Open']).apply(lambda x: x if x > 0 else 0)
etf11['loss'] = (etf11['Close']- etf11['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf11['ema_gain'] = etf11['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf11['ema_loss'] = etf11['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf11['rs'] = etf11['ema_gain'] / etf11['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf11['rsi_14'] = 100 - (100 / (etf11['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf11['range'] = etf11['High'] - etf11['Low']

# calculating the average value of ranges
etf11['atr_14'] = etf11['range'].rolling(atr_period).mean()

etf11=etf11.dropna()

In [143]:
# run the backtest
bollinger_strategy_etf11 = BBStrategy(etf11, STARTING_BALANCE, 250) 
BBresult_etf11 = bollinger_strategy_etf11.run().dropna()
BBresult_etf11['Strategy'] = 'Bollinger Bands'
BBresult_etf11

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:20,95.08,SELL,250,102.130748,86.619102,8/15/22 14:30,102.130748,-1762.687019,closed,98237.312981,Bollinger Bands
1,8/15/22 14:40,102.21,BUY,250,95.159252,110.670898,8/29/22 9:30,95.159252,-1762.687019,closed,96474.625962,Bollinger Bands
2,8/29/22 9:41,94.79,SELL,250,101.840748,86.329102,9/22/22 9:30,86.329102,2115.224423,closed,98589.850385,Bollinger Bands
3,9/22/22 9:51,85.73,SELL,250,92.780748,77.269102,10/7/22 15:13,77.269102,2115.224423,closed,100705.074808,Bollinger Bands
4,10/7/22 15:24,77.13,SELL,250,84.180748,68.669102,11/10/22 9:30,84.180748,-1762.687019,closed,98942.387789,Bollinger Bands


In [144]:
macd_trd_strategy_etf11=macd_Strategy(etf11, STARTING_BALANCE, 250)
macd_trade_etf11 = macd_trd_strategy_etf11.run().dropna()
macd_trade_etf11['Strategy'] = 'MACD'
macd_trade_etf11

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:05,94.540,BUY,250,87.489252,103.000898,6/14/22 9:43,87.489252,-1762.687019,closed,98237.312981,MACD
1,6/14/22 9:48,87.410,BUY,250,80.359252,95.870898,7/28/22 9:30,95.870898,2115.224423,closed,100352.537404,MACD
2,7/28/22 9:31,96.550,BUY,250,89.499252,105.010898,9/16/22 9:47,89.499252,-1762.687019,closed,98589.850385,MACD
3,9/16/22 9:52,89.347,BUY,250,82.296252,97.807898,9/26/22 9:34,82.296252,-1762.687019,closed,96827.163366,MACD
4,9/26/22 9:40,82.110,BUY,250,75.059252,90.570898,10/13/22 9:30,75.059252,-1762.687019,closed,95064.476347,MACD
5,10/13/22 9:37,75.000,BUY,250,67.949252,83.460898,11/1/22 9:30,83.460898,2115.224423,closed,97179.700770,MACD


In [145]:
# creating an instance of Strategy class
rsi_strategy_etf11 = RSI_Strategy(etf11, STARTING_BALANCE)

# running the backtest
RSI_run_etf11 = rsi_strategy_etf11.run()
RSI_run_etf11['Strategy'] = 'RSI'
RSI_run_etf11

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:51,94.25,BUY,250,94.028014,94.471986,5/19/22 10:53,94.471986,55.496429,closed,100055.496429,RSI
1,5/19/22 11:38,95.02,SELL,250,95.128807,94.802386,5/19/22 11:40,95.128807,-27.201786,closed,100028.294643,RSI
2,5/19/22 11:40,95.14,SELL,250,95.250593,94.918814,5/19/22 11:42,95.250593,-27.648214,closed,100000.646429,RSI
3,5/19/22 11:42,95.27,SELL,250,95.388093,95.033814,5/19/22 11:56,95.033814,59.046429,closed,100059.692857,RSI
4,5/19/22 12:44,94.34,BUY,250,94.168443,94.511557,5/19/22 12:57,94.511557,42.889286,closed,100102.582143,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
782,11/30/22 9:43,85.27,BUY,250,84.997371,85.542629,11/30/22 9:56,85.542629,68.157143,closed,99699.076786,RSI
783,11/30/22 11:01,85.39,BUY,250,85.298286,85.481714,11/30/22 11:06,85.481714,22.928571,closed,99722.005357,RSI
784,11/30/22 12:19,85.48,BUY,250,85.415671,85.544329,11/30/22 12:21,85.415671,-16.082143,closed,99705.923214,RSI
785,11/30/22 12:21,85.41,BUY,250,85.342929,85.477071,11/30/22 12:23,85.477071,16.767857,closed,99722.691071,RSI


In [146]:
# STOCH Strategy Run
STOCH_Strategy_etf11 = STOCH_Strategy(etf11, STARTING_BALANCE)
STOCH_run_etf11 = STOCH_Strategy_etf11.run()
STOCH_run_etf11['Strategy'] = 'Stochastic'
STOCH_run_etf11

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:17,94.93,SELL,250,101.980748,80.828504,8/15/22 10:29,101.980748,-1762.687019,closed,98237.312981,Stochastic
1,8/15/22 10:29,101.99,SELL,250,109.040748,87.888504,9/20/22 10:01,87.888504,3525.374038,closed,101762.687019,Stochastic
2,9/20/22 10:01,87.88,BUY,250,73.778504,101.981496,None,NaN,NaN,open,NaN,Stochastic


In [147]:
# MFI Strategy Run

mfi_strategy_etf11 = MFI_Strategy(etf11, STARTING_BALANCE)
MFI_run_etf11 = mfi_strategy_etf11.run()
MFI_run_etf11['Strategy'] = 'MFI'
MFI_run_etf11

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:31,94.568,SELL,250,101.618748,80.466504,8/12/22 15:50,101.618748,-1762.687019,closed,98237.312981,MFI
1,8/12/22 15:51,101.620,SELL,250,108.670748,87.518504,9/21/22 14:13,87.518504,3525.374038,closed,101762.687019,MFI
2,9/21/22 14:13,87.480,BUY,250,73.378504,108.632244,None,NaN,NaN,open,NaN,MFI


In [148]:
etf11_combined = pd.concat([MFI_run_etf11, RSI_run_etf11, macd_trade_etf11, BBresult_etf11], ignore_index=True, sort=True)
etf11_combined = etf11_combined.sort_values(by='open_datetime')

In [149]:
etf11_combined = etf11_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [150]:
etf11_combined['Ticker'] = 'VNQ'

In [151]:
etf11_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
566,RSI,10/10/22 12:19,10/10/22 12:15,BUY,-18.571429,closed,VNQ
567,RSI,10/10/22 12:21,10/10/22 12:19,BUY,-19.392857,closed,VNQ
568,RSI,10/10/22 12:32,10/10/22 12:21,BUY,-19.214286,closed,VNQ
569,RSI,10/10/22 12:40,10/10/22 12:32,BUY,18.678571,closed,VNQ
570,RSI,10/10/22 12:55,10/10/22 12:53,BUY,20.689286,closed,VNQ
...,...,...,...,...,...,...,...
432,RSI,9/8/22 12:04,9/8/22 12:02,BUY,-25.114286,closed,VNQ
433,RSI,9/8/22 12:05,9/8/22 12:04,BUY,-29.935714,closed,VNQ
434,RSI,9/8/22 12:10,9/8/22 12:05,BUY,-32.792857,closed,VNQ
435,RSI,9/8/22 12:32,9/8/22 12:10,BUY,49.939286,closed,VNQ


In [175]:
etf11_combined.to_csv(r'/Users/krutarth/Downloads/vnq_analysis.csv')

In [152]:
etf12 = data.iloc[:,88:96]

In [153]:
etf12 = etf12.rename(columns={'Dates.11': 'time', 'Open.11': 'Open', 'Close.11' : 'Close', 'Low.11' : 'Low', 
                            'Value.11' : 'Value', 'Volume.11' : 'Volume', 'Number Ticks.11' : 'Number Ticks', 'High.11':'High'})

In [154]:
etf12 = etf12.dropna()

In [155]:
# Generate Stichastic Oscillator 
etf12['slowk'], etf12['slowd'] = talib.STOCH(etf12['High'], etf12['Low'], etf12['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# Generate Money Flow Index
etf12['MFI'] = talib.MFI(etf12['High'], etf12['Low'], etf12['Close'], etf12['Volume'], timeperiod=14)

# generate bollinger bands
etf12['BBupper'], etf12['BBmiddle'], etf12['BBlower'] = talib.BBANDS(etf12['Close'], matype=MA_Type.T3)
etf12['sd']=etf12['Close'].std()

# Generate trade signal for BB
def trade_signal(close,lb,ub):
    if close < lb:
        return 'BUY'
    elif close > ub:
        return 'SELL'
    else:
        return 'NONE'
    
etf12['BBsignal']=np.vectorize(trade_signal)(etf12['Close'], etf12['BBlower'],etf12['BBupper'])

# Generate MACD trading signals
etf12['macd'], etf12['macdsignal'], etf12['macdhist'] = talib.MACD(etf12['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# Generate MACD Trade Signals
etf12['prev_macd']=etf12['macd'].shift(1)

def macd_signal(prev_macd, macd, macdsignal):
    if prev_macd < macdsignal and macd > macdsignal:
        return 'BUY'
    elif prev_macd > macdsignal and macd < macdsignal:
        return 'SELL'
    else:
        return 'NONE'
etf12['macd_trade_signal']=np.vectorize(trade_signal)(etf12['prev_macd'], etf12['macd'],etf12['macdsignal'])

rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
etf12['gain'] = (etf12['Close']- etf12['Open']).apply(lambda x: x if x > 0 else 0)
etf12['loss'] = (etf12['Close']- etf12['Open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
etf12['ema_gain'] = etf12['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
etf12['ema_loss'] = etf12['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
etf12['rs'] = etf12['ema_gain'] / etf12['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
etf12['rsi_14'] = 100 - (100 / (etf12['rs'] + 1))

atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
etf12['range'] = etf12['High'] - etf12['Low']

# calculating the average value of ranges
etf12['atr_14'] = etf12['range'].rolling(atr_period).mean()

etf12=etf12.dropna()

In [156]:
# run the backtest
bollinger_strategy_etf12 = BBStrategy(etf12, STARTING_BALANCE, 250) 
BBresult_etf12 = bollinger_strategy_etf12.run().dropna()
BBresult_etf12['Strategy'] = 'Bollinger Bands'
BBresult_etf12

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:09,111.010,BUY,250,100.646463,123.446245,5/31/22 9:30,123.446245,3109.061184,closed,103109.061184,Bollinger Bands
1,5/31/22 9:38,125.140,BUY,250,114.776463,137.576245,6/15/22 14:06,114.776463,-2590.884320,closed,100518.176864,Bollinger Bands
2,6/15/22 14:16,115.000,SELL,250,125.363537,102.563755,6/17/22 10:51,102.563755,3109.061184,closed,103627.238048,Bollinger Bands
3,6/17/22 10:55,102.480,SELL,250,112.843537,90.043755,8/19/22 13:10,112.843537,-2590.884320,closed,101036.353728,Bollinger Bands
4,8/19/22 13:19,112.800,BUY,250,102.436463,125.236245,9/23/22 9:36,102.436463,-2590.884320,closed,98445.469408,Bollinger Bands
5,9/23/22 9:57,101.145,SELL,250,111.508537,88.708755,10/4/22 15:49,111.508537,-2590.884320,closed,95854.585088,Bollinger Bands
6,10/4/22 15:54,111.740,SELL,250,122.103537,99.303755,10/24/22 9:37,122.103537,-2590.884320,closed,93263.700767,Bollinger Bands


In [157]:
macd_trd_strategy_etf12=macd_Strategy(etf12, STARTING_BALANCE, 250)
macd_trade_etf12 = macd_trd_strategy_etf12.run().dropna()
macd_trade_etf12['Strategy'] = 'MACD'
macd_trade_etf12

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:05,111.49,BUY,250,101.126463,123.926245,5/31/22 9:30,123.926245,3109.061184,closed,103109.061184,MACD
1,5/31/22 9:31,125.40,BUY,250,115.036463,137.836245,6/13/22 10:49,115.036463,-2590.884320,closed,100518.176864,MACD
2,6/13/22 10:51,115.16,BUY,250,104.796463,127.596245,6/17/22 10:20,104.796463,-2590.884320,closed,97927.292544,MACD
3,6/17/22 10:25,104.63,BUY,250,94.266463,117.066245,7/6/22 10:00,94.266463,-2590.884320,closed,95336.408224,MACD
4,7/6/22 10:02,94.50,BUY,250,84.136463,106.936245,7/29/22 9:30,106.936245,3109.061184,closed,98445.469408,MACD
5,7/29/22 9:31,107.63,BUY,250,97.266463,120.066245,10/20/22 10:21,120.066245,3109.061184,closed,101554.530592,MACD
6,10/20/22 10:22,120.03,SELL,250,130.393537,107.593755,11/4/22 9:35,130.393537,-2590.884320,closed,98963.646272,MACD


In [158]:
# creating an instance of Strategy class
rsi_strategy_etf12 = RSI_Strategy(etf12, STARTING_BALANCE)

# running the backtest
RSI_run_etf12 = rsi_strategy_etf12.run()
RSI_run_etf12['Strategy'] = 'RSI'
RSI_run_etf12

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 15:09,114.020,BUY,250,113.841143,114.198857,5/19/22 15:16,113.841143,-44.714286,closed,99955.285714,RSI
1,5/19/22 15:27,113.130,BUY,250,112.937286,113.322714,5/19/22 15:29,112.937286,-48.178571,closed,99907.107143,RSI
2,5/19/22 15:29,112.850,BUY,250,112.673571,113.026429,5/19/22 15:31,113.026429,44.107143,closed,99951.214286,RSI
3,5/19/22 15:34,112.800,BUY,250,112.627571,112.972429,5/19/22 15:35,112.627571,-43.107143,closed,99908.107143,RSI
4,5/19/22 15:35,112.590,BUY,250,112.394714,112.785286,5/19/22 15:40,112.785286,48.821429,closed,99956.928571,RSI
...,...,...,...,...,...,...,...,...,...,...,...,...
1302,11/30/22 15:43,127.730,BUY,250,127.627000,127.833000,11/30/22 15:46,127.627000,-25.750000,closed,99697.071429,RSI
1303,11/30/22 15:46,127.605,BUY,250,127.522143,127.687857,11/30/22 15:47,127.522143,-20.714286,closed,99676.357143,RSI
1304,11/30/22 15:47,127.518,BUY,250,127.430571,127.605429,11/30/22 15:51,127.605429,21.857143,closed,99698.214286,RSI
1305,11/30/22 15:51,127.630,BUY,250,127.504714,127.755286,11/30/22 15:52,127.504714,-31.321429,closed,99666.892857,RSI


In [159]:
# STOCH Strategy Run
STOCH_Strategy_etf12 = STOCH_Strategy(etf12, STARTING_BALANCE)
STOCH_run_etf12 = STOCH_Strategy_etf12.run()
STOCH_run_etf12['Strategy'] = 'Stochastic'
STOCH_run_etf12

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 10:19,111.73,SELL,250,122.093537,91.002925,5/27/22 11:16,122.093537,-2590.88432,closed,97409.115680,Stochastic
1,5/27/22 11:16,122.18,SELL,250,132.543537,101.452925,6/17/22 12:52,101.452925,5181.76864,closed,102590.884320,Stochastic
2,6/17/22 12:53,101.18,BUY,250,80.452925,121.907075,10/24/22 9:32,121.907075,5181.76864,closed,107772.652961,Stochastic
3,10/24/22 9:35,121.99,SELL,250,132.353537,101.262925,11/14/22 10:34,132.353537,-2590.88432,closed,105181.768640,Stochastic
4,11/14/22 10:37,132.59,SELL,250,142.953537,111.862925,None,NaN,NaN,open,NaN,Stochastic


In [160]:
# MFI Strategy Run

mfi_strategy_etf12 = MFI_Strategy(etf12, STARTING_BALANCE)
MFI_run_etf12 = mfi_strategy_etf12.run()
MFI_run_etf12['Strategy'] = 'MFI'
MFI_run_etf12

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,Strategy
0,5/19/22 11:34,112.395,SELL,250,122.758537,91.667925,5/27/22 12:54,122.758537,-2590.88432,closed,97409.11568,MFI
1,5/27/22 12:54,122.805,SELL,250,133.168537,102.077925,6/17/22 11:13,102.077925,5181.76864,closed,102590.88432,MFI
2,6/17/22 11:40,102.980,SELL,250,113.343537,82.252925,8/23/22 9:30,113.343537,-2590.88432,closed,100000.00000,MFI
3,8/23/22 9:33,114.645,SELL,250,125.008537,93.917925,10/27/22 9:30,125.008537,-2590.88432,closed,97409.11568,MFI
4,10/27/22 9:30,126.110,BUY,250,105.382925,157.200612,None,NaN,NaN,open,NaN,MFI


In [161]:
etf12_combined = pd.concat([MFI_run_etf12, RSI_run_etf12, macd_trade_etf12, BBresult_etf12], ignore_index=True, sort=True)
etf12_combined = etf12_combined.sort_values(by='open_datetime')

In [162]:
etf12_combined = etf12_combined.drop(columns=['close_price', 'open_price', 'pnl', 'sl', 'tp', 'volume'])

In [163]:
etf12_combined['Ticker'] = 'VDE'

In [164]:
etf12_combined

,Strategy,close_datetime,open_datetime,order_type,profit,status,Ticker
874,RSI,10/10/22 10:46,10/10/22 10:37,BUY,-66.785714,closed,VDE
875,RSI,10/10/22 10:52,10/10/22 10:50,BUY,-50.857143,closed,VDE
876,RSI,10/10/22 10:54,10/10/22 10:52,BUY,-49.071429,closed,VDE
877,RSI,10/10/22 10:56,10/10/22 10:54,BUY,51.321429,closed,VDE
878,RSI,10/10/22 13:01,10/10/22 13:00,BUY,-17.714286,closed,VDE
...,...,...,...,...,...,...,...
722,RSI,9/9/22 10:42,9/9/22 10:39,BUY,38.142857,closed,VDE
723,RSI,9/9/22 13:13,9/9/22 13:11,BUY,-16.607143,closed,VDE
724,RSI,9/9/22 13:15,9/9/22 13:13,BUY,20.178571,closed,VDE
725,RSI,9/9/22 14:47,9/9/22 14:39,BUY,-12.500000,closed,VDE


In [176]:
etf12_combined.to_csv(r'/Users/krutarth/Downloads/vde_analysis.csv')